# Images

## How to Make Iamges frmo Document Pages

This little script will take a document filename and generate a PNG file from each of its pages.

The document can be any supported type.

The script works as a command line tool which expects the filename being supplied as a parameter. The generated image files (1 per page) are stored in the directory of the script:

        import sys, pymupdf  # import the bindings
        fname = sys.argv[1]  # get filename from command line
        doc = pymupdf.open(fname)  # open document
        for page in doc:  # iterate through the pages
            pix = page.get_pixmap()  # render page to an image
            pix.save("page-%i.png" % page.number)  # store image as a PNG


The script directory will now contain PNG image files named page-0.png, page-1.png, etc. Pictures have the dimension of their pages with width and height rounded to integers, e.g. 595 x 842 pixels for an A4 portrait sized page. They will have a resolution of 96 dpi in x and y dimension and have no transparency. You can change all that – for how to do this, read the next sections.

In [2]:
import pymupdf 

fname = r'3-page-sample-case-report.pdf'
doc = pymupdf.open(fname)
for page in doc:
    pix = page.get_pixmap()
    print(pix)
    pix.save(f"page-{page.number}-images.png")

Pixmap(DeviceRGB, (0, 0, 612, 792), 0)
Pixmap(DeviceRGB, (0, 0, 612, 792), 0)
Pixmap(DeviceRGB, (0, 0, 612, 792), 0)


### Three images are created using the above code

    Images are created from the 3 page pdf

### How to Increase Image Resolution
The image of a document page is represented by a Pixmap, and the simplest way to create a pixmap is via method Page.get_pixmap().

This method has many options to influence the result. The most important among them is the Matrix, which lets you zoom, rotate, distort or mirror the outcome.

Page.get_pixmap() by default will use the Identity matrix, which does nothing.

In the following, we apply a zoom factor of 2 to each dimension, which will generate an image with a four times better resolution for us (and also about 4 times the size):

        zoom_x = 2.0  # horizontal zoom
        zoom_y = 2.0  # vertical zoom
        mat = pymupdf.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension
        pix = page.get_pixmap(matrix=mat)  # use 'mat' instead of the identity matrix
        
Since version 1.19.2 there is a more direct way to set the resolution: Parameter "dpi" (dots per inch) can be used in place of "matrix". To create a 300 dpi image of a page specify pix = page.get_pixmap(dpi=300). Apart from notation brevity, this approach has the additional advantage that the dpi value is saved with the image file – which does not happen automatically when using the Matrix notation.

### How to Create Partial Pixmaps (Clips)
You do not always need or want the full image of a page. This is the case e.g. when you display the image in a GUI and would like to fill the respective window with a zoomed part of the page.

Let’s assume your GUI window has room to display a full document page, but you now want to fill this room with the bottom right quarter of your page, thus using a four times better resolution.

To achieve this, define a rectangle equal to the area you want to appear in the GUI and call it “clip”. One way of constructing rectangles in PyMuPDF is by providing two diagonally opposite corners, which is what we are doing here.

        mat = pymupdf.Matrix(2, 2)  # zoom factor 2 in each direction
        rect = page.rect  # the page rectangle
        mp = (rect.tl + rect.br) / 2  # its middle point, becomes top-left of clip
        clip = pymupdf.Rect(mp, rect.br)  # the area we want
        pix = page.get_pixmap(matrix=mat, clip=clip)

In the above we construct clip by specifying two diagonally opposite points: the middle point mp of the page rectangle, and its bottom right, rect.br.

### How to Zoom a Clip to a GUI Window
Please also read the previous section. This time we want to compute the zoom factor for a clip, such that its image best fits a given GUI window. This means, that the image’s width or height (or both) will equal the window dimension. For the following code snippet you need to provide the WIDTH and HEIGHT of your GUI’s window that should receive the page’s clip rectangle.

        # WIDTH: width of the GUI window
        # HEIGHT: height of the GUI window
        # clip: a subrectangle of the document page
        # compare width/height ratios of image and window

        if clip.width / clip.height < WIDTH / HEIGHT:
            # clip is narrower: zoom to window HEIGHT
            zoom = HEIGHT / clip.height
        else:  # clip is broader: zoom to window WIDTH
            zoom = WIDTH / clip.width
        mat = pymupdf.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat, clip=clip)


For the other way round, now assume you have the zoom factor and need to compute the fitting clip.

In this case we have zoom = HEIGHT/clip.height = WIDTH/clip.width, so we must set clip.height = HEIGHT/zoom and, clip.width = WIDTH/zoom. Choose the top-left point tl of the clip on the page to compute the right pixmap:


        width = WIDTH / zoom
        height = HEIGHT / zoom
        clip = pymupdf.Rect(tl, tl.x + width, tl.y + height)
        # ensure we still are inside the page
        clip &= page.rect
        mat = pymupdf.Matrix(zoom, zoom)
        pix = pymupdf.Pixmap(matrix=mat, clip=clip)

### How to Create or Suppress Annotation Images
Normally, the pixmap of a page also shows the page’s annotations. Occasionally, this may not be desirable.

To suppress the annotation images on a rendered page, just specify annots=False in Page.get_pixmap().

You can also render annotations separately: they have their own Annot.get_pixmap() method. The resulting pixmap has the same dimensions as the annotation rectangle.

# Page
Class representing a document page. A page object is created by Document.load_page() or, equivalently, via indexing the document like doc[n] - it has no independent constructor.

There is a parent-child relationship between a document and its pages. If the document is closed or deleted, all page objects (and their respective children, too) in existence will become unusable (“orphaned”): If a page property or method is being used, an exception is raised.

Several page methods have a Document counterpart for convenience. At the end of this chapter you will find a synopsis.

## Modifying Pages
Changing page properties and adding or changing page content is available for PDF documents only.

In a nutshell, this is what you can do with PyMuPDF:

Modify page rotation and the visible part (“cropbox”) of the page.

Insert images, other PDF pages, text and simple geometrical objects.

Add annotations and form fields.

### Note

Methods require coordinates (points, rectangles) to put content in desired places. Please be aware that these coordinates must always be provided relative to the unrotated page (since v1.17.0). The reverse is also true: except Page.rect, resp. Page.bound() (both reflect when the page is rotated), all coordinates returned by methods and attributes pertain to the unrotated page.

So the returned value of e.g. Page.get_image_bbox() will not change if you do a Page.set_rotation(). The same is true for coordinates returned by Page.get_text(), annotation rectangles, and so on. If you want to find out, where an object is located in rotated coordinates, multiply the coordinates with Page.rotation_matrix. There also is its inverse, Page.derotation_matrix, which you can use when interfacing with other readers, which may behave differently in this respect.

### Note

If you add or update annotations, links or form fields on the page and immediately afterwards need to work with them (i.e. without leaving the page), you should reload the page using Document.reload_page() before referring to these new or updated items.

Reloading the page is generally recommended – although not strictly required in all cases. However, some annotation and widget types have extended features in PyMuPDF compared to MuPDF. More of these extensions may also be added in the future.

Releoading the page ensures all your changes have been fully applied to PDF structures, so you can safely create Pixmaps or successfully iterate over annotations, links and form fields.




### Method / Attribute              Short Description

                Page.add_caret_annot()          PDF only: add a caret annotation

                Page.add_circle_annot()         PDF only: add a circle annotation

                Page.add_file_annot()           PDF only: add a file attachment annotation

                Page.add_freetext_annot()       PDF only: add a text annotation

                Page.add_highlight_annot()      PDF only: add a “highlight” annotation

                Page.add_ink_annot()            PDF only: add an ink annotation

                Page.add_line_annot()           PDF only: add a line annotation

                Page.add_polygon_annot()        PDF only: add a polygon annotation

                Page.add_polyline_annot()       PDF only: add a multi-line annotation

                Page.add_rect_annot()           PDF only: add a rectangle annotation

                Page.add_redact_annot()         PDF only: add a redaction annotation

                Page.add_squiggly_annot()       PDF only: add a “squiggly” annotation

                Page.add_stamp_annot()          PDF only: add a “rubber stamp” annotation

                Page.add_strikeout_annot()      PDF only: add a “strike-out” annotation

                Page.add_text_annot()           PDF only: add a comment

                Page.add_underline_annot()      PDF only: add an “underline” annotation

                Page.add_widget()               PDF only: add a PDF Form field

                Page.annot_names()              PDF only: a list of annotation (and widget) names

                Page.annot_xrefs()              PDF only: a list of annotation (and widget) xrefs

                Page.annots()                   return a generator over the annots on the page

                Page.apply_redactions()         PDF only: process the redactions of the page

                Page.bound()                    rectangle of the page

                Page.cluster_drawings()         PDF only: bounding boxes of vector graphics

                Page.delete_annot()             PDF only: delete an annotation

                Page.delete_image()             PDF only: delete an image

                Page.delete_link()              PDF only: delete a link

                Page.delete_widget()            PDF only: delete a widget / field

                Page.draw_bezier()              PDF only: draw a cubic Bezier curve

                Page.draw_circle()              PDF only: draw a circle

                Page.draw_curve()               PDF only: draw a special Bezier curve

                Page.draw_line()                PDF only: draw a line

                Page.draw_oval()                PDF only: draw an oval / ellipse

                Page.draw_polyline()            PDF only: connect a point sequence

                Page.draw_quad()                PDF only: draw a quad

                Page.draw_rect()                PDF only: draw a rectangle

                Page.draw_sector()              PDF only: draw a circular sector

                Page.draw_squiggle()            PDF only: draw a squiggly line

                Page.draw_zigzag()              PDF only: draw a zig-zagged line

                Page.find_tables()              locate tables on the page

                Page.get_drawings()             get vector graphics on page

                Page.get_fonts()                PDF only: get list of referenced fonts

                Page.get_image_bbox()           PDF only: get bbox and matrix of embedded image

                Page.get_image_info()           get list of meta information for all used images

                Page.get_image_rects()          PDF only: improved version of Page.get_image_bbox()

                Page.get_images()               PDF only: get list of referenced images

                Page.get_label()                PDF only: return the label of the page

                Page.get_links()                get all links

                Page.get_pixmap()               create a page image in raster format

                Page.get_svg_image()            create a page image in SVG format

                Page.get_text()                 extract the page’s text

                Page.get_textbox()              extract text contained in a rectangle

                Page.get_textpage_ocr()         create a TextPage with OCR for the page

                Page.get_textpage()             create a TextPage for the page

                Page.get_xobjects()             PDF only: get list of referenced xobjects

                Page.insert_font()              PDF only: insert a font for use by the page

                Page.insert_image()             PDF only: insert an image

                Page.insert_link()              PDF only: insert a link

                Page.insert_text()              PDF only: insert text

                Page.insert_htmlbox()           PDF only: insert html text in a rectangle

                Page.insert_textbox()           PDF only: insert a text box

                Page.links()                    return a generator of the links on the page

                Page.load_annot()               PDF only: load a specific annotation

                Page.load_widget()              PDF only: load a specific field

                Page.load_links()               return the first link on a page

                Page.new_shape()                PDF only: create a new Shape

                Page.remove_rotation()          PDF only: set page rotation to 0

                Page.replace_image()            PDF only: replace an image

                Page.search_for()               search for a string

                Page.set_artbox()               PDF only: modify /ArtBox

                Page.set_bleedbox()             PDF only: modify /BleedBox

                Page.set_cropbox()              PDF only: modify the cropbox (visible page)

                Page.set_mediabox()             PDF only: modify /MediaBox

                Page.set_rotation()             PDF only: set page rotation

                Page.set_trimbox()              PDF only: modify /TrimBox

                Page.show_pdf_page()            PDF only: display PDF page image

                Page.update_link()              PDF only: modify a link

                Page.widgets()                  return a generator over the fields on the page

                Page.write_text()               write one or more TextWriter objects

                Page.cropbox_position           displacement of the cropbox

                Page.cropbox                    the page’s cropbox

                Page.artbox                     the page’s /ArtBox

                Page.bleedbox                   the page’s /BleedBox

                Page.trimbox                    the page’s /TrimBox

                Page.derotation_matrix          PDF only: get coordinates in unrotated page space

                Page.first_annot                first Annot on the page

                Page.first_link                 first Link on the page

                Page.first_widget               first widget (form field) on the page

                Page.mediabox_size              bottom-right point of mediabox

                Page.mediabox                   the page’s mediabox

                Page.number                     page number

                Page.parent                     owning document object

                Page.rect                       rectangle of the page

                Page.rotation_matrix            PDF only: get coordinates in rotated page space

                Page.rotation                   PDF only: page rotation

                Page.transformation_matrix      PDF only: translate between PDF and MuPDF space

                Page.xref                       PDF only: page xref

Class API

class Page
bound()
Determine the rectangle of the page. Same as property Page.rect. For PDF documents this usually also coincides with mediabox and cropbox, but not always. For example, if the page is rotated, then this is reflected by this method – the Page.cropbox however will not change.

RETURN TYPE:
Rect

add_caret_annot(point)
PDF only: Add a caret icon. A caret annotation is a visual symbol normally used to indicate the presence of text edits on the page.

PARAMETERS:
point (point_like) – the top left point of a 20 x 20 rectangle containing the MuPDF-provided icon.

RETURN TYPE:
Annot

RETURNS:
the created annotation. Stroke color blue = (0, 0, 1), no fill color support.

_images/img-caret-annot.jpg
Show/hide history
add_text_annot(point, text, icon='Note')
PDF only: Add a comment icon (“sticky note”) with accompanying text. Only the icon is visible, the accompanying text is hidden and can be visualized by many PDF viewers by hovering the mouse over the symbol.

PARAMETERS:
point (point_like) – the top left point of a 20 x 20 rectangle containing the MuPDF-provided “note” icon.

text (str) – the commentary text. This will be shown on double clicking or hovering over the icon. May contain any Latin characters.

icon (str) – choose one of “Note” (default), “Comment”, “Help”, “Insert”, “Key”, “NewParagraph”, “Paragraph” as the visual symbol for the embodied text [4]. (New in v1.16.0)

RETURN TYPE:
Annot

RETURNS:
the created annotation. Stroke color yellow = (1, 1, 0), no fill color support.

add_freetext_annot(rect, text, fontsize=12, fontname='helv', border_color=None, text_color=0, fill_color=1, rotate=0, align=TEXT_ALIGN_LEFT)
PDF only: Add text in a given rectangle.

PARAMETERS:
rect (rect_like) – the rectangle into which the text should be inserted. Text is automatically wrapped to a new line at box width. Lines not fitting into the box will be invisible.

text (str) – the text. May contain any mixture of Latin, Greek, Cyrillic, Chinese, Japanese and Korean characters. The respective required font is automatically determined. (New in v1.17.0)

fontsize (float) – the fontsize. Default is 12.

fontname (str) – the font name. Default is “Helv”. Accepted alternatives are “Cour”, “TiRo”, “ZaDb” and “Symb”. The name may be abbreviated to the first two characters, like “Co” for “Cour”. Lower case is also accepted. Bold or italic variants of the fonts are not accepted (changed in v1.16.0). A user-contributed script provides a circumvention for this restriction – see section Using Buttons and JavaScript in chapter FAQ. The actual font to use is now determined on a by-character level, and all required fonts (or sub-fonts) are automatically included. Therefore, you should rarely ever need to care about this parameter and let it default (except you insist on a serifed font for your non-CJK text parts). (New in v1.17.0)

text_color (sequence,float) – the text color. Default is black. (New in v1.16.0)

fill_color (sequence,float) – the fill color. Default is white. (New in v1.16.0)

text_color – the text color. Default is black.

border_color (sequence,float) – the border color. Default is None. (New in v1.19.6)

align (int) – text alignment, one of TEXT_ALIGN_LEFT, TEXT_ALIGN_CENTER, TEXT_ALIGN_RIGHT - justify is not supported. (New in v1.17.0)

rotate (int) – the text orientation. Accepted values are 0, 90, 270, invalid entries are set to zero.

RETURN TYPE:
Annot

RETURNS:
the created annotation. Color properties can only be changed using special parameters of Annot.update(). There, you can also set a border color different from the text color.

Show/hide history
add_file_annot(pos, buffer, filename, ufilename=None, desc=None, icon='PushPin')
PDF only: Add a file attachment annotation with a “PushPin” icon at the specified location.

PARAMETERS:
pos (point_like) – the top-left point of a 18x18 rectangle containing the MuPDF-provided “PushPin” icon.

buffer (bytes,bytearray,BytesIO) –

the data to be stored (actual file content, any data, etc.).

Changed in v1.14.13: io.BytesIO is now also supported.

filename (str) – the filename to associate with the data.

ufilename (str) – the optional PDF unicode version of filename. Defaults to filename.

desc (str) – an optional description of the file. Defaults to filename.

icon (str) – choose one of “PushPin” (default), “Graph”, “Paperclip”, “Tag” as the visual symbol for the attached data [4]. (New in v1.16.0)

RETURN TYPE:
Annot

RETURNS:
the created annotation. Stroke color yellow = (1, 1, 0), no fill color support.

add_ink_annot(list)
PDF only: Add a “freehand” scribble annotation.

PARAMETERS:
list (sequence) – a list of one or more lists, each containing point_like items. Each item in these sublists is interpreted as a Point through which a connecting line is drawn. Separate sublists thus represent separate drawing lines.

RETURN TYPE:
Annot

RETURNS:
the created annotation in default appearance black =(0, 0, 0),line width 1. No fill color support.

add_line_annot(p1, p2)
PDF only: Add a line annotation.

PARAMETERS:
p1 (point_like) – the starting point of the line.

p2 (point_like) – the end point of the line.

RETURN TYPE:
Annot

RETURNS:
the created annotation. It is drawn with line (stroke) color red = (1, 0, 0) and line width 1. No fill color support. The annot rectangle is automatically created to contain both points, each one surrounded by a circle of radius 3 * line width to make room for any line end symbols.

add_rect_annot(rect)
add_circle_annot(rect)
PDF only: Add a rectangle, resp. circle annotation.

PARAMETERS:
rect (rect_like) – the rectangle in which the circle or rectangle is drawn, must be finite and not empty. If the rectangle is not equal-sided, an ellipse is drawn.

RETURN TYPE:
Annot

RETURNS:
the created annotation. It is drawn with line (stroke) color red = (1, 0, 0), line width 1, fill color is supported.

Redactions
add_redact_annot(quad, text=None, fontname=None, fontsize=11, align=TEXT_ALIGN_LEFT, fill=(1, 1, 1), text_color=(0, 0, 0), cross_out=True)
PDF only: Add a redaction annotation. A redaction annotation identifies content to be removed from the document. Adding such an annotation is the first of two steps. It makes visible what will be removed in the subsequent step, Page.apply_redactions().

PARAMETERS:
quad (quad_like,rect_like) – specifies the (rectangular) area to be removed which is always equal to the annotation rectangle. This may be a rect_like or quad_like object. If a quad is specified, then the enveloping rectangle is taken.

text (str) – text to be placed in the rectangle after applying the redaction (and thus removing old content). (New in v1.16.12)

fontname (str) –

the font to use when text is given, otherwise ignored. The same rules apply as for Page.insert_textbox() – which is the method Page.apply_redactions() internally invokes. The replacement text will be vertically centered, if this is one of the CJK or PDF Base 14 Fonts. (New in v1.16.12)

Note

For an existing font of the page, use its reference name as fontname (this is item[4] of its entry in Page.get_fonts()).

For a new, non-builtin font, proceed as follows:

page.insert_text(point,  # anywhere, but outside all redaction rectangles
    "something",  # some non-empty string
    fontname="newname",  # new, unused reference name
    fontfile="...",  # desired font file
    render_mode=3,  # makes the text invisible
)
page.add_redact_annot(..., fontname="newname")
fontsize (float) – the fontsize to use for the replacing text. If the text is too large to fit, several insertion attempts will be made, gradually reducing the fontsize to no less than 4. If then the text will still not fit, no text insertion will take place at all. (New in v1.16.12)

align (int) – the horizontal alignment for the replacing text. See insert_textbox() for available values. The vertical alignment is (approximately) centered if a PDF built-in font is used (CJK or PDF Base 14 Fonts). (New in v1.16.12)

fill (sequence) – the fill color of the rectangle after applying the redaction. The default is white = (1, 1, 1), which is also taken if None is specified. To suppress a fill color altogether, specify False. In this cases the rectangle remains transparent. (New in v1.16.12)

text_color (sequence) – the color of the replacing text. Default is black = (0, 0, 0). (New in v1.16.12)

cross_out (bool) – add two diagonal lines to the annotation rectangle. (New in v1.17.2)

RETURN TYPE:
Annot

RETURNS:
the created annotation. Its standard appearance looks like a red rectangle (no fill color), optionally showing two diagonal lines. Colors, line width, dashing, opacity and blend mode can now be set and applied via Annot.update() like with other annotations. (Changed in v1.17.2)

_images/img-redact.jpg
Show/hide history
apply_redactions(images=PDF_REDACT_IMAGE_PIXELS | 2, graphics=PDF_REDACT_LINE_ART_IF_TOUCHED | 2, text=PDF_REDACT_TEXT_REMOVE | 0)
PDF only: Remove all content contained in any redaction rectangle on the page.

This method applies and then deletes all redactions from the page.

PARAMETERS:
images (int) – How to redact overlapping images. The default (2) blanks out overlapping pixels. PDF_REDACT_IMAGE_NONE | 0 ignores, and PDF_REDACT_IMAGE_REMOVE | 1 completely removes images overlapping any redaction annotation. Option PDF_REDACT_IMAGE_REMOVE_UNLESS_INVISIBLE | 3 only removes images that are actually visible.

graphics (int) – How to redact overlapping vector graphics (also called “line-art” or “drawings”). The default (2) removes any overlapping vector graphics. PDF_REDACT_LINE_ART_NONE | 0 ignores, and PDF_REDACT_LINE_ART_IF_COVERED | 1 removes graphics fully contained in a redaction annotation. When removing line-art, please be aware that stroked vector graphics (i.e. type “s” or “sf”) have a larger wrapping rectangle than one might expect: first of all, at least 50% of the path’s line width have to be added in each direction to truly include all of the drawing. If a so-called “miter limit” is provided (see page 121 of the PDF specification), the enlarging value is miter * width / 2. So, when letting everything default (width = 1, miter = 10), the redaction rectangle should be at least 5 points larger in every direction.

text (int) – Whether to redact overlapping text. The default PDF_REDACT_TEXT_REMOVE | 0 removes all characters whose boundary box overlaps any redaction rectangle. This complies with the original legal / data protection intentions of redaction annotations. Other use cases however may require to keep text while redacting vector graphics or images. This can be achieved by setting text=True|PDF_REDACT_TEXT_NONE | 1. This does not comply with the data protection intentions of redaction annotations. Do so at your own risk.

RETURNS:
True if at least one redaction annotation has been processed, False otherwise.

Note

Text contained in a redaction rectangle will be physically removed from the page (assuming Document.save() with a suitable garbage option) and will no longer appear in e.g. text extractions or anywhere else. All redaction annotations will also be removed. Other annotations are unaffected.

All overlapping links will be removed. If the rectangle of the link was covering text, then only the overlapping part of the text is being removed. Similar applies to images covered by link rectangles.

The overlapping parts of images will be blanked-out for default option PDF_REDACT_IMAGE_PIXELS (changed in v1.18.0). Option 0 does not touch any images and 1 will remove any image with an overlap.

For option images=PDF_REDACT_IMAGE_REMOVE only this page’s references to the images are removed - not necessarily the images themselves. Images are completely removed from the file only, if no longer referenced at all (assuming suitable garbage collection options).

For option images=PDF_REDACT_IMAGE_PIXELS a new image of format PNG is created, which the page will use in place of the original one. The original image is not deleted or replaced as part of this process, so other pages may still show the original. In addition, the new, modified PNG image currently is stored uncompressed. Do keep these aspects in mind when choosing the right garbage collection method and compression options during save.

Text removal is done by character: A character is removed if its bbox has a non-empty overlap with a redaction rectangle (changed in MuPDF v1.17). Depending on the font properties and / or the chosen line height, deletion may occur for undesired text parts. Using Tools.set_small_glyph_heights() with a True argument before text search may help to prevent this.

Redactions are a simple way to replace single words in a PDF, or to just physically remove them. Locate the word “secret” using some text extraction or search method and insert a redaction using “xxxxxx” as replacement text for each occurrence.

Be wary if the replacement is longer than the original – this may lead to an awkward appearance, line breaks or no new text at all.

For a number of reasons, the new text may not exactly be positioned on the same line like the old one – especially true if the replacement font was not one of CJK or PDF Base 14 Fonts.

Show/hide history
add_polyline_annot(points)
add_polygon_annot(points)
PDF only: Add an annotation consisting of lines which connect the given points. A Polygon’s first and last points are automatically connected, which does not happen for a PolyLine. The rectangle is automatically created as the smallest rectangle containing the points, each one surrounded by a circle of radius 3 (= 3 * line width). The following shows a ‘PolyLine’ that has been modified with colors and line ends.

PARAMETERS:
points (list) – a list of point_like objects.

RETURN TYPE:
Annot

RETURNS:
the created annotation. It is drawn with line color black, line width 1 no fill color but fill color support. Use methods of Annot to make any changes to achieve something like this:

_images/img-polyline.png
add_underline_annot(quads=None, start=None, stop=None, clip=None)
add_strikeout_annot(quads=None, start=None, stop=None, clip=None)
add_squiggly_annot(quads=None, start=None, stop=None, clip=None)
add_highlight_annot(quads=None, start=None, stop=None, clip=None)
PDF only: These annotations are normally used for marking text which has previously been somehow located (for example via Page.search_for()). But this is not required: you are free to “mark” just anything.

Standard (stroke only – no fill color support) colors are chosen per annotation type: yellow for highlighting, red for striking out, green for underlining, and magenta for wavy underlining.

All these four methods convert the arguments into a list of Quad objects. The annotation rectangle is then calculated to envelop all these quadrilaterals.

Note

search_for() delivers a list of either Rect or Quad objects. Such a list can be directly used as an argument for these annotation types and will deliver one common annotation for all occurrences of the search string:

# prefer quads=True in text searching for annotations!
quads = page.search_for("pymupdf", quads=True)
page.add_highlight_annot(quads)
Note

Obviously, text marker annotations need to know what is the top, the bottom, the left, and the right side of the area(s) to be marked. If the arguments are quads, this information is given by the sequence of the quad points. In contrast, a rectangle delivers much less information – this is illustrated by the fact, that 4! = 24 different quads can be constructed with the four corners of a rectangle.

Therefore, we strongly recommend to use the quads option for text searches, to ensure correct annotations. A similar consideration applies to marking text spans extracted with the “dict” / “rawdict” options of Page.get_text(). For more details on how to compute quadrilaterals in this case, see section “How to Mark Non-horizontal Text” of FAQ.

PARAMETERS:
quads (rect_like,quad_like,list,tuple) – the location(s) – rectangle(s) or quad(s) – to be marked. (Changed in v1.14.20) A list or tuple must consist of rect_like or quad_like items (or even a mixture of either). Every item must be finite, convex and not empty (as applicable). Set this parameter to None if you want to use the following arguments (Changed in v1.16.14). And vice versa: if not None, the remaining parameters must be None.

start (point_like) – start text marking at this point. Defaults to the top-left point of clip. Must be provided if quads is None. (New in v1.16.14)

stop (point_like) – stop text marking at this point. Defaults to the bottom-right point of clip. Must be used if quads is None. (New in v1.16.14)

clip (rect_like) – only consider text lines intersecting this area. Defaults to the page rectangle. Only use if start and stop are provided. (New in v1.16.14)

RETURN TYPE:
Annot or None (changed in v1.16.14).

RETURNS:
the created annotation. If quads is an empty list, no annotation is created (changed in v1.16.14).

Note

You can use parameters start, stop and clip to highlight consecutive lines between the points start and stop (starting with v1.16.14). Make use of clip to further reduce the selected line bboxes and thus deal with e.g. multi-column pages. The following multi-line highlight on a page with three text columns was created by specifying the two red points and setting clip accordingly.

_images/img-markers.jpg
cluster_drawings(clip=None, drawings=None, x_tolerance=3, y_tolerance=3)
Cluster vector graphics (synonyms are line-art or drawings) based on their geometrical vicinity. The method walks through the output of Page.get_drawings() and joins paths whose path["rect"] are closer to each other than some tolerance values (given in the arguments). The result is a list of rectangles that each wrap things like tables (with gridlines), pie charts, bar charts, etc.

PARAMETERS:
clip (rect_like) – only consider paths inside this area. The default is the full page.

drawings (list) – (optional) provide a previously generated output of Page.get_drawings(). If None the method will execute the method.

x_tolerance (float) –

find_tables(clip=None, strategy=None, vertical_strategy=None, horizontal_strategy=None, vertical_lines=None, horizontal_lines=None, snap_tolerance=None, snap_x_tolerance=None, snap_y_tolerance=None, join_tolerance=None, join_x_tolerance=None, join_y_tolerance=None, edge_min_length=3, min_words_vertical=3, min_words_horizontal=1, intersection_tolerance=None, intersection_x_tolerance=None, intersection_y_tolerance=None, text_tolerance=None, text_x_tolerance=None, text_y_tolerance=None, add_lines=None)
Find tables on the page and return an object with related information. Typically, the default values of the many parameters will be sufficient. Adjustments should ever only be needed in corner case situations.

PARAMETERS:
clip (rect_like) – specify a region to consider within the page rectangle and ignore the rest. Default is the full page.

strategy (str) –

Request a table detection strategy. Valid values are “lines”, “lines_strict” and “text”.

Default is “lines” which uses all vector graphics on the page to detect grid lines.

Strategy “lines_strict” ignores borderless rectangle vector graphics. Sometimes single text pieces have background colors which may lead to false columns or lines. This strategy ignores them and can thus increase detection precision.

If “text” is specified, text positions are used to generate “virtual” column and / or row boundaries. Use min_words_* to request the number of words for considering their coordinates.

Use parameters vertical_strategy and horizontal_strategy instead for a more fine-grained treatment of the dimensions.

horizontal_lines (sequence[floats]) – y-coordinates of rows. If provided, there will be no attempt to identify additional table rows. This influences table detection.

vertical_lines (sequence[floats]) – x-coordinates of columns. If provided, there will be no attempt to identify additional table columns. This influences table detection.

min_words_vertical (int) – relevant for vertical strategy option “text”: at least this many words must coincide to establish a virtual column boundary.

min_words_horizontal (int) – relevant for horizontal strategy option “text”: at least this many words must coincide to establish a virtual row boundary.

snap_tolerance (float) – Any two horizontal lines whose y-values differ by no more than this value will be snapped into one. Accordingly for vertical lines. Default is 3. Separate values can be specified instead for the dimensions, using snap_x_tolerance and snap_y_tolerance.

join_tolerance (float) – Any two lines will be joined to one if the end and the start points differ by no more than this value (in points). Default is 3. Instead of this value, separate values can be specified for the dimensions using join_x_tolerance and join_y_tolerance.

edge_min_length (float) – Ignore a line if its length does not exceed this value (points). Default is 3.

intersection_tolerance (float) – When combining lines into cell borders, orthogonal lines must be within this value (points) to be considered intersecting. Default is 3. Instead of this value, separate values can be specified for the dimensions using intersection_x_tolerance and intersection_y_tolerance.

text_tolerance (float) – Characters will be combined into words only if their distance is no larger than this value (points). Default is 3. Instead of this value, separate values can be specified for the dimensions using text_x_tolerance and text_y_tolerance.

add_lines (tuple,list) – Specify a list of “lines” (i.e. pairs of point_like objects) as additional, “virtual” vector graphics. These lines may help with table and / or cell detection and will not otherwise influence the detection strategy. Especially, in contrast to parameters horizontal_lines and vertical_lines, they will not prevent detecting rows or columns in other ways. These lines will be treated exactly like “real” vector graphics in terms of joining, snapping, intersectiing, minimum length and containment in the clip rectangle. Similarly, lines not parallel to any of the coordinate axes will be ignored.

_images/img-findtables.jpg
RETURNS:
a TableFinder object that has the following significant attributes:

cells: a list of all bboxes on the page, that have been identified as table cells (across all tables). Each cell is a rect_like tuple (x0, y0, x1, y1) of coordinates or None.

tables: a list of Table objects. This is [] if the page has no tables. Single tables can be found as items of this list. But the TableFinder object itself is also a sequence of its tables. This means that if tabs is a TableFinder object, then table “n” is delivered by tabs.tables[n] as well as by the shorter tabs[n].

The Table object has the following attributes:

bbox: the bounding box of the table as a tuple (x0, y0, x1, y1).

cells: bounding boxes of the table’s cells (list of tuples). A cell may also be None.

extract(): this method returns the text content of each table cell as a list of list of strings.

to_markdown(): this method returns the table as a string in markdown format (compatible to Github). Supporting viewers can render the string as a table. This output is optimized for small token sizes, which is especially beneficial for LLM/RAG feeds. Pandas DataFrames (see method to_pandas() below) offer an equivalent markdown table output which however is better readable for the human eye.

to_pandas(): this method returns the table as a pandas DataFrame. DataFrames are very versatile objects allowing a plethora of table manipulation methods and outputs to almost 20 well-known formats, among them Excel files, CSV, JSON, markdown-formatted tables and more. DataFrame.to_markdown() generates a Github-compatible markdown format optimized for human readability. This method however requires the package [tablutate](https://pypi.org/project/tabulate/) to installed in addition to pandas itself.

header: a TableHeader object containing header information of the table.

col_count: an integer containing the number of table columns.

row_count: an integer containing the number of table rows.

rows: a list of TableRow objects containing two attributes, bbox is the boundary box of the row, and cells is a list of table cells contained in this row.

The TableHeader object has the following attributes:

bbox: the bounding box of the header.

cells: a list of bounding boxes containing the name of the respective column.

names: a list of strings containing the text of each of the cell bboxes. They represent the column names – which are used when exporting the table to pandas DataFrames, markdown, etc.

external: a bool indicating whether the header bbox is outside the table body (True) or not. Table headers are never identified by the TableFinder logic. Therefore, if external is true, then the header cells are not part of any cell identified by TableFinder. If external == False, then the first table row is the header.

Please have a look at these Jupyter notebooks, which cover standard situations like multiple tables on one page or joining table fragments across multiple pages.

Show/hide history
Important

There is also the pdf2docx extract tables method which is capable of table extraction if you prefer.

add_stamp_annot(rect, stamp=0)
PDF only: Add a “rubber stamp” like annotation to e.g. indicate the document’s intended use (“DRAFT”, “CONFIDENTIAL”, etc.).

PARAMETERS:
rect (rect_like) – rectangle where to place the annotation.

stamp (int) – id number of the stamp text. For available stamps see Stamp Annotation Icons.

Note

The stamp’s text and its border line will automatically be sized and be put horizontally and vertically centered in the given rectangle. Annot.rect is automatically calculated to fit the given width and will usually be smaller than this parameter.

The font chosen is “Times Bold” and the text will be upper case.

The appearance can be changed using Annot.set_opacity() and by setting the “stroke” color (no “fill” color supported).

This can be used to create watermark images: on a temporary PDF page create a stamp annotation with a low opacity value, make a pixmap from it with alpha=True (and potentially also rotate it), discard the temporary PDF page and use the pixmap with insert_image() for your target PDF.

_images/img-stampannot.jpg
add_widget(widget)
PDF only: Add a PDF Form field (“widget”) to a page. This also turns the PDF into a Form PDF. Because of the large amount of different options available for widgets, we have developed a new class Widget, which contains the possible PDF field attributes. It must be used for both, form field creation and updates.

PARAMETERS:
widget (Widget) – a Widget object which must have been created upfront.

RETURNS:
a widget annotation.

delete_annot(annot)
The removal will now include any bound ‘Popup’ or response annotations and related objects (changed in v1.16.6).

PDF only: Delete annotation from the page and return the next one.

PARAMETERS:
annot (Annot) – the annotation to be deleted.

RETURN TYPE:
Annot

RETURNS:
the annotation following the deleted one. Please remember that physical removal requires saving to a new file with garbage > 0.

delete_widget(widget)
PDF only: Delete field from the page and return the next one.

PARAMETERS:
widget (Widget) – the widget to be deleted.

RETURN TYPE:
Widget

RETURNS:
the widget following the deleted one. Please remember that physical removal requires saving to a new file with garbage > 0.

Show/hide history
delete_link(linkdict)
PDF only: Delete the specified link from the page. The parameter must be an original item of get_links(), see Description of get_links() Entries. The reason for this is the dictionary’s “xref” key, which identifies the PDF object to be deleted.

PARAMETERS:
linkdict (dict) – the link to be deleted.

insert_link(linkdict)
PDF only: Insert a new link on this page. The parameter must be a dictionary of format as provided by get_links(), see Description of get_links() Entries.

PARAMETERS:
linkdict (dict) – the link to be inserted.

update_link(linkdict)
PDF only: Modify the specified link. The parameter must be a (modified) original item of get_links(), see Description of get_links() Entries. The reason for this is the dictionary’s “xref” key, which identifies the PDF object to be changed.

PARAMETERS:
linkdict (dict) – the link to be modified.

Warning

If updating / inserting a URI link ("kind": LINK_URI), please make sure to start the value for the "uri" key with a disambiguating string like "http://", "https://", "file://", "ftp://", "mailto:", etc. Otherwise – depending on your browser or other “consumer” software – unexpected default assumptions may lead to unwanted behaviours.

get_label()
PDF only: Return the label for the page.

RETURN TYPE:
str

RETURNS:
the label string like “vii” for Roman numbering or “” if not defined.

Show/hide history
get_links()
Retrieves all links of a page.

RETURN TYPE:
list

RETURNS:
A list of dictionaries. For a description of the dictionary entries, see Description of get_links() Entries. Always use this or the Page.links() method if you intend to make changes to the links of a page.

links(kinds=None)
Return a generator over the page’s links. The results equal the entries of Page.get_links().

PARAMETERS:
kinds (sequence) – a sequence of integers to down-select to one or more link kinds. Default is all links. Example: kinds=(pymupdf.LINK_GOTO,) will only return internal links.

RETURN TYPE:
generator

RETURNS:
an entry of Page.get_links() for each iteration.

Show/hide history
annots(types=None)
Return a generator over the page’s annotations.

PARAMETERS:
types (sequence) – a sequence of integers to down-select to one or more annotation types. Default is all annotations. Example: types=(pymupdf.PDF_ANNOT_FREETEXT, pymupdf.PDF_ANNOT_TEXT) will only return ‘FreeText’ and ‘Text’ annotations.

RETURN TYPE:
generator

RETURNS:
an Annot for each iteration.

Caution

You cannot safely update annotations from within this generator. This is because most annotation updates require reloading the page via page = doc.reload_page(page). To circumvent this restriction, make a list of annotations xref numbers first and then iterate over these numbers:

In [4]: xrefs = [annot.xref for annot in page.annots(types=[...])]
In [5]: for xref in xrefs:
   ...:     annot = page.load_annot(xref)
   ...:     annot.update()
   ...:     page = doc.reload_page(page)
In [6]:
Show/hide history
widgets(types=None)
Return a generator over the page’s form fields.

PARAMETERS:
types (sequence) – a sequence of integers to down-select to one or more widget types. Default is all form fields. Example: types=(pymupdf.PDF_WIDGET_TYPE_TEXT,) will only return ‘Text’ fields.

RETURN TYPE:
generator

RETURNS:
a Widget for each iteration.

Show/hide history
write_text(rect=None, writers=None, overlay=True, color=None, opacity=None, keep_proportion=True, rotate=0, oc=0)
PDF only: Write the text of one or more TextWriter objects to the page.

PARAMETERS:
rect (rect_like) – where to place the text. If omitted, the rectangle union of the text writers is used.

writers (sequence) – a non-empty tuple / list of TextWriter objects or a single TextWriter.

opacity (float) – set transparency, overwrites resp. value in the text writers.

color (sequ) – set the text color, overwrites resp. value in the text writers.

overlay (bool) – put the text in foreground or background.

keep_proportion (bool) – maintain the aspect ratio.

rotate (float) – rotate the text by an arbitrary angle.

oc (int) – the xref of an OCG or OCMD. (New in v1.18.4)

Note

Parameters overlay, keep_proportion, rotate and oc have the same meaning as in Page.show_pdf_page().

Show/hide history
insert_text(point, text, fontsize=11, fontname='helv', fontfile=None, idx=0, color=None, fill=None, render_mode=0, border_width=1, encoding=TEXT_ENCODING_LATIN, rotate=0, morph=None, stroke_opacity=1, fill_opacity=1, overlay=True, oc=0)
PDF only: Insert text starting at point_like point. See Shape.insert_text().

Show/hide history
insert_textbox(rect, buffer, fontsize=11, fontname='helv', fontfile=None, idx=0, color=None, fill=None, render_mode=0, border_width=1, encoding=TEXT_ENCODING_LATIN, expandtabs=8, align=TEXT_ALIGN_LEFT, charwidths=None, rotate=0, morph=None, stroke_opacity=1, fill_opacity=1, oc=0, overlay=True)
PDF only: Insert text into the specified rect_like rect. See Shape.insert_textbox().

Show/hide history
insert_htmlbox(rect, text, *, css=None, scale_low=0, archive=None, rotate=0, oc=0, opacity=1, overlay=True)
PDF only: Insert text into the specified rectangle. The method has similarities with methods Page.insert_textbox() and TextWriter.fill_textbox(), but is much more powerful. This is achieved by letting a Story object do all the required processing.

Parameter text may be a string as in the other methods. But it will be interpreted as HTML source and may therefore also contain HTML language elements – including styling. The css parameter may be used to pass in additional styling instructions.

Automatic line breaks are generated at word boundaries. The “soft hyphen” character "&#173;" (or &shy;) can be used to cause hyphenation and thus may also cause line breaks. Forced line breaks however are only achievable via the HTML tag <br> - "\n" is ignored and will be treated like a space.

With this method the following can be achieved:

Styling effects like bold, italic, text color, text alignment, font size or font switching.

The text may include arbitrary languages – including right-to-left languages.

Scripts like Devanagari and several others in Asia have a highly complex system of ligatures, where two or more unicodes together yield one glyph. The Story uses the software package HarfBuzz , to deal with these things and produce correct output.

One can also include images via HTML tag <img> – the Story will take care of the appropriate layout. This is an alternative option to insert images, compared to Page.insert_image().

HTML tables (tag <table>) may be included in the text and will be handled appropriately.

Links are automatically generated when present.

If content does not fit in the rectangle, the developer has two choices:

either only be informed about this (and accept a no-op, just like with the other textbox insertion methods),

or (scale_low=0 - the default) scale down the content until it fits.

PARAMETERS:
rect (rect_like) – rectangle on page to receive the text.

text (str,Story) – the text to be written. Can contain a mixture of plain text and HTML tags with styling instructions. Alternatively, a Story object may be specified (in which case the internal Story generation step will be omitted). A Story must have been generated with all required styling and Archive information.

css (str) – optional string containing additional CSS instructions. This parameter is ignored if text is a Story.

scale_low (float) – if necessary, scale down the content until it fits in the target rectangle. This sets the down scaling limit. Default is 0, no limit. A value of 1 means no down-scaling permitted. A value of e.g. 0.2 means maximum down-scaling by 80%.

archive (Archive) – an Archive object that points to locations where to find images or non-standard fonts. If text refers to images or non-standard fonts, this parameter is required. This parameter is ignored if text is a Story.

rotate (int) –

one of the values 0, 90, 180, 270. Depending on this, text will be filled:

0: top-left to bottom-right.

90: bottom-left to top-right.

180: bottom-right to top-left.

270: top-right to bottom-left.

_images/img-rotate.png
oc (int) – the xref of an OCG / OCMD or 0. Please refer to Page.show_pdf_page() for details.

opacity (float) – set the fill and stroke opacity of the content. Only values 0 <= opacity < 1 are considered.

overlay (bool) – put the text in front of other content. Please refer to Page.show_pdf_page() for details.

RETURNS:
A tuple of floats (spare_height, scale).

spare_height: -1 if content did not fit, else >= 0. It is the height of the unused (still available) rectangle stripe. Positive only if scale = 1 (no down-scaling happened).

scale: down-scaling factor, 0 < scale <= 1.

Please refer to examples in this section of the recipes: How to Fill a Box with HTML Text.

Show/hide history
Drawing Methods

draw_line(p1, p2, color=(0,), width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a line from p1 to p2 (point_like s). See Shape.draw_line().

Show/hide history
draw_zigzag(p1, p2, breadth=2, color=(0,), width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a zigzag line from p1 to p2 (point_like s). See Shape.draw_zigzag().

Show/hide history
draw_squiggle(p1, p2, breadth=2, color=(0,), width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a squiggly (wavy, undulated) line from p1 to p2 (point_like s). See Shape.draw_squiggle().

Show/hide history
draw_circle(center, radius, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a circle around center (point_like) with a radius of radius. See Shape.draw_circle().

Show/hide history
draw_oval(quad, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw an oval (ellipse) within the given rect_like or quad_like. See Shape.draw_oval().

Show/hide history
draw_sector(center, point, angle, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, fullSector=True, overlay=True, closePath=False, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a circular sector, optionally connecting the arc to the circle’s center (like a piece of pie). See Shape.draw_sector().

Show/hide history
draw_polyline(points, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, closePath=False, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw several connected lines defined by a sequence of point_like s. See Shape.draw_polyline().

Show/hide history
draw_bezier(p1, p2, p3, p4, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, closePath=False, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a cubic Bézier curve from p1 to p4 with the control points p2 and p3 (all are point_like s). See Shape.draw_bezier().

Show/hide history
draw_curve(p1, p2, p3, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, closePath=False, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: This is a special case of draw_bezier(). See Shape.draw_curve().

Show/hide history
draw_rect(rect, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, radius=None, oc=0)
PDF only: Draw a rectangle. See Shape.draw_rect().

Show/hide history
draw_quad(quad, color=(0,), fill=None, width=1, dashes=None, lineCap=0, lineJoin=0, overlay=True, morph=None, stroke_opacity=1, fill_opacity=1, oc=0)
PDF only: Draw a quadrilateral. See Shape.draw_quad().

Show/hide history
insert_font(fontname='helv', fontfile=None, fontbuffer=None, set_simple=False, encoding=TEXT_ENCODING_LATIN)
PDF only: Add a new font to be used by text output methods and return its xref. If not already present in the file, the font definition will be added. Supported are the built-in Base14_Fonts and the CJK fonts via “reserved” fontnames. Fonts can also be provided as a file path or a memory area containing the image of a font file.

PARAMETERS:
fontname (str) –

The name by which this font shall be referenced when outputting text on this page. In general, you have a “free” choice here (but consult the Adobe PDF References, page 16, section 7.3.5 for a formal description of building legal PDF names). However, if it matches one of the Base14_Fonts or one of the CJK fonts, fontfile and fontbuffer are ignored.

In other words, you cannot insert a font via fontfile / fontbuffer and also give it a reserved fontname.

Note

A reserved fontname can be specified in any mixture of upper or lower case and still match the right built-in font definition: fontnames “helv”, “Helv”, “HELV”, “Helvetica”, etc. all lead to the same font definition “Helvetica”. But from a Page perspective, these are different references. You can exploit this fact when using different encoding variants (Latin, Greek, Cyrillic) of the same font on a page.

fontfile (str) – a path to a font file. If used, fontname must be different from all reserved names.

fontbuffer (bytes/bytearray) – the memory image of a font file. If used, fontname must be different from all reserved names. This parameter would typically be used with Font.buffer for fonts supported / available via Font.

set_simple (int) – applicable for fontfile / fontbuffer cases only: enforce treatment as a “simple” font, i.e. one that only uses character codes up to 255.

encoding (int) – applicable for the “Helvetica”, “Courier” and “Times” sets of Base14_Fonts only. Select one of the available encodings Latin (0), Cyrillic (2) or Greek (1). Only use the default (0 = Latin) for “Symbol” and “ZapfDingBats”.

RYTPE:
int

RETURNS:
the xref of the installed font.

Note

Built-in fonts will not lead to the inclusion of a font file. So the resulting PDF file will remain small. However, your PDF viewer software is responsible for generating an appropriate appearance – and there exist differences on whether or how each one of them does this. This is especially true for the CJK fonts. But also Symbol and ZapfDingbats are incorrectly handled in some cases. Following are the Font Names and their correspondingly installed Base Font names:

Base-14 Fonts [1]

Font Name

Installed Base Font

Comments

helv

Helvetica

normal

heit

Helvetica-Oblique

italic

hebo

Helvetica-Bold

bold

hebi

Helvetica-BoldOblique

bold-italic

cour

Courier

normal

coit

Courier-Oblique

italic

cobo

Courier-Bold

bold

cobi

Courier-BoldOblique

bold-italic

tiro

Times-Roman

normal

tiit

Times-Italic

italic

tibo

Times-Bold

bold

tibi

Times-BoldItalic

bold-italic

symb

Symbol

[3]

zadb

ZapfDingbats

[3]

CJK Fonts [2] (China, Japan, Korea)

Font Name

Installed Base Font

Comments

china-s

Heiti

simplified Chinese

china-ss

Song

simplified Chinese (serif)

china-t

Fangti

traditional Chinese

china-ts

Ming

traditional Chinese (serif)

japan

Gothic

Japanese

japan-s

Mincho

Japanese (serif)

korea

Dotum

Korean

korea-s

Batang

Korean (serif)

insert_image(rect, *, alpha=-1, filename=None, height=0, keep_proportion=True, mask=None, oc=0, overlay=True, pixmap=None, rotate=0, stream=None, width=0, xref=0)
PDF only: Put an image inside the given rectangle. The image may already exist in the PDF or be taken from a pixmap, a file, or a memory area.

PARAMETERS:
rect (rect_like) – where to put the image. Must be finite and not empty.

alpha (int) – deprecated and ignored.

filename (str) – name of an image file (all formats supported by MuPDF – see Supported Input Image Formats).

height (int) –

keep_proportion (bool) – maintain the aspect ratio of the image.

mask (bytes,bytearray,io.BytesIO) – image in memory – to be used as image mask (alpha values) for the base image. When specified, the base image must be provided as a filename or a stream – and must not be an image that already has a mask.

oc (int) – (xref) make image visibility dependent on this OCG or OCMD. Ignored after the first of multiple insertions. The property is stored with the generated PDF image object and therefore controls the image’s visibility throughout the PDF.

overlay – see Common Parameters.

pixmap (Pixmap) – a pixmap containing the image.

rotate (int) – rotate the image. Must be an integer multiple of 90 degrees. Positive values rotate anti-clockwise. If you need a rotation by an arbitrary angle, consider converting the image to a PDF (Document.convert_to_pdf()) first and then use Page.show_pdf_page() instead.

stream (bytes,bytearray,io.BytesIO) – image in memory (all formats supported by MuPDF – see Supported Input Image Formats).

width (int) –

xref (int) – the xref of an image already present in the PDF. If given, parameters filename, Pixmap, stream, alpha and mask are ignored. The page will simply receive a reference to the existing image.

RETURNS:
The xref of the embedded image. This can be used as the xref argument for very significant performance boosts, if the image is inserted again.

This example puts the same image on every page of a document:

doc = pymupdf.open(...)
rect = pymupdf.Rect(0, 0, 50, 50)       # put thumbnail in upper left corner
img = open("some.jpg", "rb").read()  # an image file
img_xref = 0                         # first execution embeds the image
for page in doc:
      img_xref = page.insert_image(rect, stream=img,
                 xref=img_xref,  2nd time reuses existing image
          )
doc.save(...)
Note

The method detects multiple insertions of the same image (like in the above example) and will store its data only on the first execution. This is even true (although less performant), if using the default xref=0.

The method cannot detect if the same image had already been part of the file before opening it.

You can use this method to provide a background or foreground image for the page, like a copyright or a watermark. Please remember, that watermarks require a transparent image if put in foreground …

The image may be inserted uncompressed, e.g. if a Pixmap is used or if the image has an alpha channel. Therefore, consider using deflate=True when saving the file. In addition, there are ways to control the image size – even if transparency comes into play. Have a look at How to Add Images to a PDF Page.

The image is stored in the PDF at its original quality level. This may be much better than what you need for your display. Consider decreasing the image size before insertion – e.g. by using the pixmap option and then shrinking it or scaling it down (see Pixmap chapter). The PIL method Image.thumbnail() can also be used for that purpose. The file size savings can be very significant.

Another efficient way to display the same image on multiple pages is another method: show_pdf_page(). Consult Document.convert_to_pdf() for how to obtain intermediary PDFs usable for that method.

Show/hide history
replace_image(xref, filename=None, pixmap=None, stream=None)
Replace the image at xref with another one.

PARAMETERS:
xref (int) – the xref of the image.

filename – the filename of the new image.

pixmap – the Pixmap of the new image.

stream – the memory area containing the new image.

Arguments filename, Pixmap, stream have the same meaning as in Page.insert_image(), especially exactly one of these must be provided.

This is a global replacement: the new image will also be shown wherever the old one has been displayed throughout the file.

This method mainly exists for technical purposes. Typical uses include replacing large images by smaller versions, like a lower resolution, graylevel instead of colored, etc., or changing transparency.

Show/hide history
delete_image(xref)
Delete the image at xref. This is slightly misleading: actually the image is being replaced with a small transparent Pixmap using above Page.replace_image(). The visible effect however is equivalent.

PARAMETERS:
xref (int) – the xref of the image.

This is a global replacement: the image will disappear wherever the old one has been displayed throughout the file.

If you inspect / extract a page’s images by methods like Page.get_images(), Page.get_image_info() or Page.get_text(), the replacing “dummy” image will be detected like so (45, 47, 1, 1, 8, 'DeviceGray', '', 'Im1', 'FlateDecode') and also seem to “cover” the same boundary box on the page.

Show/hide history
get_text(option, *, clip=None, flags=None, textpage=None, sort=False, delimiters=None)
Retrieves the content of a page in a variety of formats. This is a wrapper for multiple TextPage methods by choosing the output option opt as follows:

“text” – TextPage.extractTEXT(), default

“blocks” – TextPage.extractBLOCKS()

“words” – TextPage.extractWORDS()

“html” – TextPage.extractHTML()

“xhtml” – TextPage.extractXHTML()

“xml” – TextPage.extractXML()

“dict” – TextPage.extractDICT()

“json” – TextPage.extractJSON()

“rawdict” – TextPage.extractRAWDICT()

“rawjson” – TextPage.extractRAWJSON()

PARAMETERS:
opt (str) –

A string indicating the requested format, one of the above. A mixture of upper and lower case is supported.

Values “words” and “blocks” are also accepted (changed in v1.16.3).

clip (rect-like) – restrict extracted text to this rectangle. If None, the full page is taken. Has no effect for options “html”, “xhtml” and “xml”. (New in v1.17.7)

flags (int) – indicator bits to control whether to include images or how text should be handled with respect to white spaces and ligatures. See Text Extraction Flags for available indicators and Text Extraction Flags Defaults for default settings. (New in v1.16.2)

textpage – use a previously created TextPage. This reduces execution time very significantly: by more than 50% and up to 95%, depending on the extraction option. If specified, the ‘flags’ and ‘clip’ arguments are ignored, because they are textpage-only properties. If omitted, a new, temporary textpage will be created. (New in v1.19.0)

sort (bool) – sort the output by vertical, then horizontal coordinates. In many cases, this should suffice to generate a “natural” reading order. Has no effect on (X)HTML and XML. Output option “words” sorts by (y1, x0) of the words’ bboxes. Similar is true for “blocks”, “dict”, “json”, “rawdict”, “rawjson”: they all are sorted by (y1, x0) of the resp. block bbox. If specified for “text”, then internally “blocks” is used. (New in v1.19.1)

delimiters (str) – use these characters as additional word separators with the “words” output option (ignored otherwise). By default, all white spaces (including non-breaking space 0xA0) indicate start and end of a word. Now you can specify more characters causing this. For instance, the default will return "john.doe@outlook.com" as one word. If you specify delimiters="@." then the four words "john", "doe", "outlook", "com" will be returned. Other possible uses include ignoring punctuation characters delimiters=string.punctuation. The “word” strings will not contain any delimiting character. (New in v1.23.5)

RETURN TYPE:
str, list, dict

RETURNS:
The page’s content as a string, a list or a dictionary. Refer to the corresponding TextPage method for details.

Note

You can use this method as a document conversion tool from any supported document type to one of TEXT, HTML, XHTML or XML documents.

The inclusion of text via the clip parameter is decided on a by-character level: a character becomes part of the output, if its bbox is contained in clip (changed in v1.18.2). This deviates from the algorithm used in redaction annotations: a character will be removed if its bbox intersects any redaction annotation.

Show/hide history
get_textbox(rect, textpage=None)
Retrieve the text contained in a rectangle.

PARAMETERS:
rect (rect-like) – rect-like.

textpage – a TextPage to use. If omitted, a new, temporary textpage will be created.

RETURNS:
a string with interspersed linebreaks where necessary. It is based on dedicated code (changed in v1.19.0). A typical use is checking the result of Page.search_for():

rl = page.search_for("currency:")
page.get_textbox(rl[0])
'Currency:'

Show/hide history
get_textpage(clip=None, flags=3)
Create a TextPage for the page.

PARAMETERS:
flags (int) – indicator bits controlling the content available for subsequent text extractions and searches – see the parameter of Page.get_text().

clip (rect-like) – restrict extracted text to this area. (New in v1.17.7)

RETURNS:
TextPage

Show/hide history
get_textpage_ocr(flags=3, language='eng', dpi=72, full=False, tessdata=None)
Optical Character Recognition (OCR) technology can be used to extract text data for documents where text is in a raster image format throughout the page. Use this method to OCR a page for text extraction.

This method returns a TextPage for the page that includes OCRed text. MuPDF will invoke Tesseract-OCR if this method is used. Otherwise this is a normal TextPage object.

PARAMETERS:
flags (int) – indicator bits controlling the content available for subsequent test extractions and searches – see the parameter of Page.get_text().

language (str) – the expected language(s). Use “+”-separated values if multiple languages are expected, “eng+spa” for English and Spanish.

dpi (int) – the desired resolution in dots per inch. Influences recognition quality (and execution time).

full (bool) – whether to OCR the full page, or just the displayed images.

tessdata (str) – The name of Tesseract’s language support folder tessdata. If omitted, this information must be present as environment variable TESSDATA_PREFIX. Can be determined by function get_tessdata().

Note

This method does not support a clip parameter – OCR will always happen for the complete page rectangle.

RETURNS:
a TextPage. Execution may be significantly longer than Page.get_textpage().

For a full page OCR, all text will have the font “GlyphlessFont” from Tesseract. In case of partial OCR, normal text will keep its properties, and only text coming from images will have the GlyphlessFont.

Note

OCRed text is only available to PyMuPDF’s text extractions and searches if their TextPage parameter specifies the output of this method.

This Jupyter notebook walks through an example for using OCR textpages.

Show/hide history
get_drawings(extended=False)
Return the vector graphics of the page. These are instructions which draw lines, rectangles, quadruples or curves, including properties like colors, transparency, line width and dashing, etc. Alternative terms are “line art” and “drawings”.

RETURNS:
a list of dictionaries. Each dictionary item contains one or more single draw commands belonging together: they have the same properties (colors, dashing, etc.). This is called a “path” in PDF, so we adopted that name here, but the method works for all document types.

The path dictionary for fill, stroke and fill-stroke paths has been designed to be compatible with class Shape. There are the following keys:

Key

Value

closePath

Same as the parameter in Shape.

color

Stroke color (see Shape).

dashes

Dashed line specification (see Shape).

even_odd

Fill colors of area overlaps – same as the parameter in Shape.

fill

Fill color (see Shape).

items

List of draw commands: lines, rectangles, quads or curves.

lineCap

Number 3-tuple, use its max value on output with Shape.

lineJoin

Same as the parameter in Shape.

fill_opacity

fill color transparency (see Shape). (New in v1.18.17)

stroke_opacity

stroke color transparency (see Shape). (New in v1.18.17)

rect

Page area covered by this path. Information only.

layer

name of applicable Optional Content Group. (New in v1.22.0)

level

the hierarchy level if extended=True. (New in v1.22.0)

seqno

command number when building page appearance. (New in v1.19.0)

type

type of this path. (New in v1.18.17)

width

Stroke line width. (see Shape).

Key "opacity" has been replaced by the new keys "fill_opacity" and "stroke_opacity". This is now compatible with the corresponding parameters of Shape.finish(). (Changed in v1.18.17)

For paths other than groups or clips, key "type" takes one of the following values:

“f” – this is a fill-only path. Only key-values relevant for this operation have a meaning, not applicable ones are present with a value of None: "color", "lineCap", "lineJoin", "width", "closePath", "dashes" and should be ignored.

“s” – this is a stroke-only path. Similar to previous, key "fill" is present with value None.

“fs” – this is a path performing combined fill and stroke operations.

Each item in path["items"] is one of the following:

("l", p1, p2) - a line from p1 to p2 (Point objects).

("c", p1, p2, p3, p4) - cubic Bézier curve from p1 to p4 (p2 and p3 are the control points). All objects are of type Point.

("re", rect, orientation) - a Rect. Multiple rectangles within the same path are now detected (changed in v1.18.17). Integer orientation is 1 resp. -1 indicating whether the enclosed area is rotated left (1 = anti-clockwise), or resp. right [7] (changed in v1.19.2).

("qu", quad) - a Quad. 3 or 4 consecutive lines are detected to actually represent a Quad (changed in v1.19.2:). (New in v1.18.17)

Using class Shape, you should be able to recreate the original drawings on a separate (PDF) page with high fidelity under normal, not too sophisticated circumstances. Please see the following comments on restrictions. A coding draft can be found in section “Extractings Drawings” of chapter FAQ.

Specifying extended=True significantly alters the output. Most importantly, new dictionary types are present: “clip” and “group”. All paths will now be organized in a hierarchic structure which is encoded by the new integer key “level”, the hierarchy level. Each group or clip establishes a new hierarchy, which applies to all subsequent paths having a larger level value. (New in v1.22.0)

Any path with a smaller level value than its predecessor will end the scope of (at least) the preceeding hierarchy level. A “clip” path with the same level as the preceding clip will end the scope of that clip. Same is true for groups. This is best explained by an example:

+------+------+--------+------+--------+
| line | lvl0 | lvl1   | lvl2 |  lvl3  |
+------+------+--------+------+--------+
|  0   | clip |        |      |        |
|  1   |      | fill   |      |        |
|  2   |      | group  |      |        |
|  3   |      |        | clip |        |
|  4   |      |        |      | stroke |
|  5   |      |        | fill |        |  ends scope of clip in line 3
|  6   |      | stroke |      |        |  ends scope of group in line 2
|  7   |      | clip   |      |        |
|  8   | fill |        |      |        |  ends scope of line 0
+------+------+--------+------+--------+
The clip in line 0 applies to line including line 7. Group in line 2 applies to lines 3 to 5, clip in line 3 only applies to line 4.

“stroke” in line 4 is under control of “group” in line 2 and “clip” in line 3 (which in turn is a subset of line 0 clip).

“clip” dictionary. Its values (most importantly “scissor”) remain valid / apply as long as following dictionaries have a larger “level” value.

Key

Value

closePath

Same as in “stroke” or “fill” dictionaries

even_odd

Same as in “stroke” or “fill” dictionaries

items

Same as in “stroke” or “fill” dictionaries

rect

Same as in “stroke” or “fill” dictionaries

layer

Same as in “stroke” or “fill” dictionaries

level

Same as in “stroke” or “fill” dictionaries

scissor

the clip rectangle

type

“clip”

“group” dictionary. Its values remain valid (apply) as long as following dictionaries have a larger “level” value. Any dictionary with an equal or lower level end this group.

Key

Value

rect

Same as in “stroke” or “fill” dictionaries

layer

Same as in “stroke” or “fill” dictionaries

level

Same as in “stroke” or “fill” dictionaries

isolated

(bool) Whether this group is isolated

knockout

(bool) Whether this is a “Knockout Group”

blendmode

Name of the BlendMode, default is “Normal”

opacity

Float value in range [0, 1].

type

“group”

Note

The method is based on the output of Page.get_cdrawings() – which is much faster, but requires somewhat more attention processing its output.

Show/hide history
get_cdrawings(extended=False)
Extract the vector graphics on the page. Apart from following technical differences, functionally equivalent to Page.get_drawings(), but much faster:

Every path type only contains the relevant keys, e.g. a stroke path has no "fill" color key. See comment in method Page.get_drawings().

Coordinates are given as point_like, rect_like and quad_like tuples – not as Point, Rect, Quad objects.

If performance is a concern, consider using this method: Compared to versions earlier than 1.18.17, you should see much shorter response times. We have seen pages that required 2 seconds then, now only need 200 ms with this method.

Show/hide history
get_fonts(full=False)
PDF only: Return a list of fonts referenced by the page. Wrapper for Document.get_page_fonts().

get_images(full=False)
PDF only: Return a list of images referenced by the page. Wrapper for Document.get_page_images().

get_image_info(hashes=False, xrefs=False)
Return a list of meta information dictionaries for all images shown on the page. This works for all document types. Technically, this is a subset of the dictionary output of Page.get_text(): the image binary content and any text on the page are ignored.

PARAMETERS:
hashes (bool) – Compute the MD5 hashcode for each encountered image, which allows identifying image duplicates. This adds the key "digest" to the output, whose value is a 16 byte bytes object. (New in v1.18.13)

xrefs (bool) – PDF only. Try to find the xref for each image. Implies hashes=True. Adds the "xref" key to the dictionary. If not found, the value is 0, which means, the image is either “inline” or otherwise undetectable. Please note that this option has an extended response time, because the MD5 hashcode will be computed at least two times for each image with an xref. (New in v1.18.13)

RETURN TYPE:
list[dict]

RETURNS:
A list of dictionaries. This includes information for exactly those images, that are shown on the page – including “inline images”. In contrast to images included in Page.get_text(), image binary content is not loaded, which drastically reduces memory usage. The dictionary layout is similar to that of image blocks in page.get_text("dict").

Key

Value

number

block number (int)

bbox

image bbox on page, rect_like

width

original image width (int)

height

original image height (int)

cs-name

colorspace name (str)

colorspace

colorspace.n (int)

xres

resolution in x-direction (int)

yres

resolution in y-direction (int)

bpc

bits per component (int)

size

storage occupied by image (int)

digest

MD5 hashcode (bytes), if hashes is true

xref

image xref or 0, if xrefs is true

transform

matrix transforming image rect to bbox, matrix_like

Multiple occurrences of the same image are always reported. You can detect duplicates by comparing their digest values.

Show/hide history
get_xobjects()
PDF only: Return a list of Form XObjects referenced by the page. Wrapper for Document.get_page_xobjects().

get_image_rects(item, transform=False)
PDF only: Return boundary boxes and transformation matrices of an embedded image. This is an improved version of Page.get_image_bbox() with the following differences:

There is no restriction on how the image is invoked (by the page or one of its Form XObjects). The result is always complete and correct.

The result is a list of Rect or (Rect, Matrix) objects – depending on transform. Each list item represents one location of the image on the page. Multiple occurrences might not be detectable by Page.get_image_bbox().

The method invokes Page.get_image_info() with xrefs=True and therefore has a noticeably longer response time than Page.get_image_bbox().

PARAMETERS:
item (list,str,int) – an item of the list Page.get_images(), or the reference name entry of such an item (item[7]), or the image xref.

transform (bool) – also return the matrix used to transform the image rectangle to the bbox on the page. If true, then tuples (bbox, matrix) are returned.

RETURN TYPE:
list

RETURNS:
Boundary boxes and respective transformation matrices for each image occurrence on the page. If the item is not on the page, an empty list [] is returned.

Show/hide history
get_image_bbox(item, transform=False)
PDF only: Return boundary box and transformation matrix of an embedded image.

PARAMETERS:
item (list,str) – an item of the list Page.get_images() with full=True specified, or the reference name entry of such an item, which is item[-3] (or item[7] respectively).

transform (bool) – return the matrix used to transform the image rectangle to the bbox on the page (new in v1.18.11). Default is just the bbox. If true, then a tuple (bbox, matrix) is returned.

RETURN TYPE:
Rect or (Rect, Matrix)

RETURNS:
the boundary box of the image – optionally also its transformation matrix.

Show/hide history
Note

Be aware that Page.get_images() may contain “dead” entries i.e. images, which the page does not display. This is no error, but intended by the PDF creator. No exception will be raised in this case, but an infinite rectangle is returned. You can avoid this from happening by executing Page.clean_contents() before this method.

The image’s “transformation matrix” is defined as the matrix, for which the expression bbox / transform == pymupdf.Rect(0, 0, 1, 1) is true, lookup details here: Image Transformation Matrix.

Show/hide history
get_svg_image(matrix=pymupdf.Identity, text_as_path=True)
Create an SVG image from the page. Only full page images are currently supported.

PARAMETERS:
matrix (matrix_like) – a matrix, default is Identity.

text_as_path (bool) – – controls how text is represented. True outputs each character as a series of elementary draw commands, which leads to a more precise text display in browsers, but a very much larger output for text-oriented pages. Display quality for False relies on the presence of the referenced fonts on the current system. For missing fonts, the internet browser will fall back to some default – leading to unpleasant appearances. Choose False if you want to parse the text of the SVG. (New in v1.17.5)

RETURNS:
a UTF-8 encoded string that contains the image. Because SVG has XML syntax it can be saved in a text file, the standard extension is .svg.

Note

In case of a PDF, you can circumvent the “full page image only” restriction by modifying the page’s CropBox before using the method.

get_pixmap(*, matrix=pymupdf.Identity, dpi=None, colorspace=pymupdf.csRGB, clip=None, alpha=False, annots=True)
Create a pixmap from the page. This is probably the most often used method to create a Pixmap.

All parameters are keyword-only.

PARAMETERS:
matrix (matrix_like) – default is Identity.

dpi (int) – desired resolution in x and y direction. If not None, the "matrix" parameter is ignored. (New in v1.19.2)

colorspace (str or Colorspace) – The desired colorspace, one of “GRAY”, “RGB” or “CMYK” (case insensitive). Or specify a Colorspace, ie. one of the predefined ones: csGRAY, csRGB or csCMYK.

clip (irect_like) – restrict rendering to the intersection of this area with the page’s rectangle.

alpha (bool) –

whether to add an alpha channel. Always accept the default False if you do not really need transparency. This will save a lot of memory (25% in case of RGB … and pixmaps are typically large!), and also processing time. Also note an important difference in how the image will be rendered: with True the pixmap’s samples area will be pre-cleared with 0x00. This results in transparent areas where the page is empty. With False the pixmap’s samples will be pre-cleared with 0xff. This results in white where the page has nothing to show.

Show/hide history
annots (bool) – (new in version 1.16.0) whether to also render annotations or to suppress them. You can create pixmaps for annotations separately.

RETURN TYPE:
Pixmap

RETURNS:
Pixmap of the page. For fine-controlling the generated image, the by far most important parameter is matrix. E.g. you can increase or decrease the image resolution by using Matrix(xzoom, yzoom). If zoom > 1, you will get a higher resolution: zoom=2 will double the number of pixels in that direction and thus generate a 2 times larger image. Non-positive values will flip horizontally, resp. vertically. Similarly, matrices also let you rotate or shear, and you can combine effects via e.g. matrix multiplication. See the Matrix section to learn more.

Note

The pixmap will have “premultiplied” pixels if alpha=True. To learn about some background, e.g. look for “Premultiplied alpha” here.

The method will respect any page rotation and will not exceed the intersection of clip and Page.cropbox. If you need the page’s mediabox (and if this is a different rectangle), you can use a snippet like the following to achieve this:

In [1]: import pymupdf
In [2]: doc=pymupdf.open("demo1.pdf")
In [3]: page=doc[0]
In [4]: rotation = page.rotation
In [5]: cropbox = page.cropbox
In [6]: page.set_cropbox(page.mediabox)
In [7]: page.set_rotation(0)
In [8]: pix = page.get_pixmap()
In [9]: page.set_cropbox(cropbox)
In [10]: if rotation != 0:
   ...:     page.set_rotation(rotation)
   ...:
In [11]:
Show/hide history
annot_names()
PDF only: return a list of the names of annotations, widgets and links. Technically, these are the /NM values of every PDF object found in the page’s /Annots array.

RETURN TYPE:
list

Show/hide history
annot_xrefs()
PDF only: return a list of the :data`xref` numbers of annotations, widgets and links – technically of all entries found in the page’s /Annots array.

RETURN TYPE:
list

RETURNS:
a list of items (xref, type) where type is the annotation type. Use the type to tell apart links, fields and annotations, see Annotation Types.

Show/hide history
load_annot(ident)
PDF only: return the annotation identified by ident. This may be its unique name (PDF /NM key), or its xref.

PARAMETERS:
ident (str,int) – the annotation name or xref.

RETURN TYPE:
Annot

RETURNS:
the annotation or None.

Note

Methods Page.annot_names(), Page.annot_xrefs() provide lists of names or xrefs, respectively, from where an item may be picked and loaded via this method.

Show/hide history
load_widget(xref)
PDF only: return the field identified by xref.

PARAMETERS:
xref (int) – the field’s xref.

RETURN TYPE:
Widget

RETURNS:
the field or None.

Note

This is similar to the analogous method Page.load_annot() – except that here only the xref is supported as identifier.

Show/hide history
load_links()
Return the first link on a page. Synonym of property first_link.

RETURN TYPE:
Link

RETURNS:
first link on the page (or None).

set_rotation(rotate)
PDF only: Set the rotation of the page.

PARAMETERS:
rotate (int) – An integer specifying the required rotation in degrees. Must be an integer multiple of 90. Values will be converted to one of 0, 90, 180, 270.

remove_rotation()
PDF only: Set page rotation to 0 while maintaining appearance and page content.

RETURNS:
The inverted matrix used to achieve this change. If the page was not rotated (rotation 0), Identity is returned. The method automatically recomputes the rectangles of any annotations, links and widgets present on the page.

This method may come in handy when e.g. used with Page.show_pdf_page().

show_pdf_page(rect, docsrc, pno=0, keep_proportion=True, overlay=True, oc=0, rotate=0, clip=None)
PDF only: Display a page of another PDF as a vector image (otherwise similar to Page.insert_image()). This is a multi-purpose method. For example, you can use it to

create “n-up” versions of existing PDF files, combining several input pages into one output page (see example combine.py),

create “posterized” PDF files, i.e. every input page is split up in parts which each create a separate output page (see posterize.py),

include PDF-based vector images like company logos, watermarks, etc., see svg-logo.py, which puts an SVG-based logo on each page (requires additional packages to deal with SVG-to-PDF conversions).

PARAMETERS:
rect (rect_like) – where to place the image on current page. Must be finite and its intersection with the page must not be empty.

docsrc (Document) – source PDF document containing the page. Must be a different document object, but may be the same file.

pno (int) – page number (0-based, in -∞ < pno < docsrc.page_count) to be shown.

keep_proportion (bool) – whether to maintain the width-height-ratio (default). If false, all 4 corners are always positioned on the border of the target rectangle – whatever the rotation value. In general, this will deliver distorted and /or non-rectangular images.

overlay (bool) – put image in foreground (default) or background.

oc (int) – (xref) make visibility dependent on this OCG / OCMD (which must be defined in the target PDF) [9]. (New in v1.18.3)

rotate (float) – show the source rectangle rotated by some angle. Any angle is supported (changed in v1.14.11). (New in v1.14.10)

clip (rect_like) – choose which part of the source page to show. Default is the full page, else must be finite and its intersection with the source page must not be empty.

Note

In contrast to method Document.insert_pdf(), this method does not copy annotations, widgets or links, so these are not included in the target [6]. But all its other resources (text, images, fonts, etc.) will be imported into the current PDF. They will therefore appear in text extractions and in get_fonts() and get_images() lists – even if they are not contained in the visible area given by clip.

Example: Show the same source page, rotated by 90 and by -90 degrees:

doc = pymupdf.open()  # new empty PDF
page=doc.new_page()  # new page in A4 format

# upper half page
r1 = pymupdf.Rect(0, 0, page.rect.width, page.rect.height/2)

# lower half page
r2 = r1 + (0, page.rect.height/2, 0, page.rect.height/2)

src = pymupdf.open("PyMuPDF.pdf")  # show page 0 of this

page.show_pdf_page(r1, src, 0, rotate=90)
page.show_pdf_page(r2, src, 0, rotate=-90)
doc.save("show.pdf")
_images/img-showpdfpage.jpg
Show/hide history
new_shape()
PDF only: Create a new Shape object for the page.

RETURN TYPE:
Shape

RETURNS:
a new Shape to use for compound drawings. See description there.

search_for(needle, *, clip=None, quads=False, flags=TEXT_DEHYPHENATE | TEXT_PRESERVE_WHITESPACE | TEXT_PRESERVE_LIGATURES | TEXT_MEDIABOX_CLIP, textpage=None)
Search for needle on a page. Wrapper for TextPage.search().

PARAMETERS:
needle (str) – Text to search for. May contain spaces. Upper / lower case is ignored, but only works for ASCII characters: For example, “COMPÉTENCES” will not be found if needle is “compétences” – “compÉtences” however will. Similar is true for German umlauts and the like.

clip (rect_like) – only search within this area. (New in v1.18.2)

quads (bool) – Return object type Quad instead of Rect.

flags (int) – Control the data extracted by the underlying TextPage. By default, ligatures and white spaces are kept, and hyphenation [8] is detected.

textpage – use a previously created TextPage. This reduces execution time significantly. If specified, the ‘flags’ and ‘clip’ arguments are ignored. If omitted, a temporary textpage will be created. (New in v1.19.0)

RETURN TYPE:
list

RETURNS:
A list of Rect or Quad objects, each of which – normally! – surrounds one occurrence of needle. However: if parts of needle occur on more than one line, then a separate item is generated for each these parts. So, if needle = "search string", two rectangles may be generated.

Show/hide history
Note

The method supports multi-line text marker annotations: you can use the full returned list as one single parameter for creating the annotation.

Caution

There is a tricky aspect: the search logic regards contiguous multiple occurrences of needle as one: assuming needle is “abc”, and the page contains “abc” and “abcabc”, then only two rectangles will be returned, one for “abc”, and a second one for “abcabc”.

You can always use Page.get_textbox() to check what text actually is being surrounded by each rectangle.

Note

A feature repeatedly asked for is supporting regular expressions when specifying the "needle" string: There is no way to do this. If you need something in that direction, first extract text in the desired format and then subselect the result by matching with some regex pattern. Here is an example for matching words:

pattern = re.compile(r"...")  # the regex pattern
words = page.get_text("words")  # extract words on page
matches = [w for w in words if pattern.search(w[4])]
The matches list will contain the words matching the given pattern. In the same way you can select span["text"] from the output of page.get_text("dict").

Show/hide history
set_mediabox(r)
PDF only: Change the physical page dimension by setting mediabox in the page’s object definition.

PARAMETERS:
r (rect-like) – the new mediabox value.

Note

This method also removes the page’s other (optional) rectangles (cropbox, ArtBox, TrimBox and Bleedbox) to prevent inconsistent situations. This will cause those to assume their default values.

Caution

For non-empty pages this may have undesired effects, because the location of all content depends on this value and will therefore change position or even disappear.

Show/hide history
set_cropbox(r)
PDF only: change the visible part of the page.

PARAMETERS:
r (rect_like) – the new visible area of the page. Note that this must be specified in unrotated coordinates, not empty, nor infinite and be completely contained in the Page.mediabox.

After execution (if the page is not rotated), Page.rect will equal this rectangle, but be shifted to the top-left position (0, 0) if necessary. Example session:

page = doc.new_page()
page.rect
pymupdf.Rect(0.0, 0.0, 595.0, 842.0)

page.cropbox  # cropbox and mediabox still equal
pymupdf.Rect(0.0, 0.0, 595.0, 842.0)

# now set cropbox to a part of the page
page.set_cropbox(pymupdf.Rect(100, 100, 400, 400))
# this will also change the "rect" property:
page.rect
pymupdf.Rect(0.0, 0.0, 300.0, 300.0)

# but mediabox remains unaffected
page.mediabox
pymupdf.Rect(0.0, 0.0, 595.0, 842.0)

# revert CropBox change
# either set it to MediaBox
page.set_cropbox(page.mediabox)
# or 'refresh' MediaBox: will remove all other rectangles
page.set_mediabox(page.mediabox)
set_artbox(r)
set_bleedbox(r)
set_trimbox(r)
PDF only: Set the resp. rectangle in the page object. For the meaning of these objects see Adobe PDF References, page 77. Parameter and restrictions are the same as for Page.set_cropbox().

Show/hide history
rotation
Contains the rotation of the page in degrees (always 0 for non-PDF types). This is a copy of the value in the PDF file. The PDF documentation says:

“The number of degrees by which the page should be rotated clockwise when displayed or printed. The value must be a multiple of 90. Default value: 0.”

In PyMuPDF, we make sure that this attribute is always one of 0, 90, 180 or 270.

TYPE:
int

cropbox_position
Contains the top-left point of the page’s /CropBox for a PDF, otherwise Point(0, 0).

TYPE:
Point

cropbox
The page’s /CropBox for a PDF. Always the unrotated page rectangle is returned. For a non-PDF this will always equal the page rectangle.

Note

In PDF, the relationship between /MediaBox, /CropBox and page rectangle may sometimes be confusing, please do lookup the glossary for MediaBox.

TYPE:
Rect

artbox
bleedbox
trimbox
The page’s /ArtBox, /BleedBox, /TrimBox, respectively. If not provided, defaulting to Page.cropbox.

TYPE:
Rect

mediabox_size
Contains the width and height of the page’s Page.mediabox for a PDF, otherwise the bottom-right coordinates of Page.rect.

TYPE:
Point

mediabox
The page’s mediabox for a PDF, otherwise Page.rect.

TYPE:
Rect

Note

For most PDF documents and for all other document types, page.rect == page.cropbox == page.mediabox is true. However, for some PDFs the visible page is a true subset of mediabox. Also, if the page is rotated, its Page.rect may not equal Page.cropbox. In these cases the above attributes help to correctly locate page elements.

transformation_matrix
This matrix translates coordinates from the PDF space to the MuPDF space. For example, in PDF /Rect [x0 y0 x1 y1] the pair (x0, y0) specifies the bottom-left point of the rectangle – in contrast to MuPDF’s system, where (x0, y0) specify top-left. Multiplying the PDF coordinates with this matrix will deliver the (Py-) MuPDF rectangle version. Obviously, the inverse matrix will again yield the PDF rectangle.

TYPE:
Matrix

rotation_matrix
derotation_matrix
These matrices may be used for dealing with rotated PDF pages. When adding / inserting anything to a PDF page, the coordinates of the unrotated page are always used. These matrices help translating between the two states. Example: if a page is rotated by 90 degrees – what would then be the coordinates of the top-left Point(0, 0) of an A4 page?

page.set_rotation(90)  # rotate an ISO A4 page
page.rect
Rect(0.0, 0.0, 842.0, 595.0)
p = pymupdf.Point(0, 0)  # where did top-left point land?
p * page.rotation_matrix
Point(842.0, 0.0)

TYPE:
Matrix

first_link
Contains the first Link of a page (or None).

TYPE:
Link

first_annot
Contains the first Annot of a page (or None).

TYPE:
Annot

first_widget
Contains the first Widget of a page (or None).

TYPE:
Widget

number
The page number.

TYPE:
int

parent
The owning document object.

TYPE:
Document

rect
Contains the rectangle of the page. Same as result of Page.bound().

TYPE:
Rect

xref
The page’s PDF xref. Zero if not a PDF.

TYPE:
Rect



### Homologous Methods of Document and Page
This is an overview of homologous methods on the Document and on the Page level.

### Document Level              Page Level

        Document.get_page_fonts(pno)        Page.get_fonts()

        Document.get_page_images(pno)       Page.get_images()

        Document.get_page_pixmap(pno, …)    Page.get_pixmap()

        Document.get_page_text(pno, …)      Page.get_text()

        Document.search_page_for(pno, …)    Page.search_for()

The page number “pno” is a 0-based integer -∞ < pno < page_count.

### Annot

This class is for PDF only.
Quote from the Adobe PDF References: “An annotation associates an object such as a note, sound, or movie with a location on a page of a PDF document, or provides a way to interact with the user by means of the mouse and keyboard.”

There is a parent-child relationship between an annotation and its page. If the page object becomes unusable (closed document, any document structure change, etc.), then so does every of its existing annotation objects – an exception is raised saying that the object is “orphaned”, whenever an annotation property or method is accessed.

### Attribute                   Short Description

        Annot.delete_responses()    delete all responding annotions

        Annot.get_file()            get attached file content

        Annot.get_oc()              get xref of an OCG / OCMD

        Annot.get_pixmap()          image of the annotation as a pixmap

        Annot.get_sound()           get the sound of an audio annotation

        Annot.get_text()            extract annotation text

        Annot.get_textbox()         extract annotation text

        Annot.set_border()          set annotation’s border properties

        Annot.set_blendmode()       set annotation’s blend mode

        Annot.set_colors()          set annotation’s colors

        Annot.set_flags()           set annotation’s flags field

        Annot.set_irt_xref()        define the annotation to being “In Response To”

        Annot.set_name()            set annotation’s name field

        Annot.set_oc()              set xref to an OCG / OCMD

        Annot.set_opacity()         change transparency

        Annot.set_open()            open / close annotation or its Popup

        Annot.set_popup()           create a Popup for the annotation

        Annot.set_rect()            change annotation rectangle

        Annot.set_rotation()        change rotation

        Annot.update_file()         update attached file content

        Annot.update()              apply accumulated annot changes

        Annot.blendmode             annotation BlendMode

        Annot.border                border details

        Annot.colors                border / background and fill colors

        Annot.file_info             get attached file information

        Annot.flags                 annotation flags

        Annot.has_popup             whether annotation has a Popup

        Annot.irt_xref              annotation to which this one responds

        Annot.info                  various information

        Annot.is_open               whether annotation or its Popup is open

        Annot.line_ends             start / end appearance of line-type annotations

        Annot.next                  link to the next annotation

        Annot.opacity               the annot’s transparency

        Annot.parent                page object of the annotation

        Annot.popup_rect            rectangle of the annotation’s Popup

        Annot.popup_xref            the PDF xref number of the annotation’s Popup

        Annot.rect                  rectangle containing the annotation

        Annot.type                  type of the annotation

        Annot.vertices              point coordinates of Polygons, PolyLines, etc.

        Annot.xref                  the PDF xref number

### Class API

class Annot
get_pixmap(matrix=pymupdf.Identity, dpi=None, colorspace=pymupdf.csRGB, alpha=False)
Changed in v1.19.2: added support of dpi parameter.

Creates a pixmap from the annotation as it appears on the page in untransformed coordinates. The pixmap’s IRect equals Annot.rect.irect (see below). All parameters are keyword only.

PARAMETERS:
matrix (matrix_like) – a matrix to be used for image creation. Default is Identity.

dpi (int) – (new in v1.19.2) desired resolution in dots per inch. If not None, the matrix parameter is ignored.

colorspace (Colorspace) – a colorspace to be used for image creation. Default is pymupdf.csRGB.

alpha (bool) – whether to include transparency information. Default is False.

RETURN TYPE:
Pixmap

Note

If the annotation has just been created or modified, you should Document.reload_page() the page first via page = doc.reload_page(page).

The pixmap will have “premultiplied” pixels if alpha=True. To learn about some background, e.g. look for “Premultiplied alpha” here.

get_text(opt, clip=None, flags=None)
New in 1.18.0

Retrieves the content of the annotation in a variety of formats – much like the same method for Page.. This currently only delivers relevant data for annotation types ‘FreeText’ and ‘Stamp’. Other types return an empty string (or equivalent objects).

PARAMETERS:
opt (str) –

(positional only) the desired format - one of the following values. Please note that this method works exactly like the same-named method of Page.

”text” – TextPage.extractTEXT(), default

”blocks” – TextPage.extractBLOCKS()

”words” – TextPage.extractWORDS()

”html” – TextPage.extractHTML()

”xhtml” – TextPage.extractXHTML()

”xml” – TextPage.extractXML()

”dict” – TextPage.extractDICT()

”json” – TextPage.extractJSON()

”rawdict” – TextPage.extractRAWDICT()

clip (rect-like) – (keyword only) restrict the extraction to this area. Should hardly ever be required, defaults to Annot.rect.

flags (int) – (keyword only) control the amount of data returned. Defaults to simple text extraction.

get_textbox(rect)
New in 1.18.0

Return the annotation text. Mostly (except line breaks) equal to Annot.get_text() with the “text” option.

PARAMETERS:
rect (rect-like) – the area to consider, defaults to Annot.rect.

set_info(info=None, content=None, title=None, creationDate=None, modDate=None, subject=None)
Changed in version 1.16.10

Changes annotation properties. These include dates, contents, subject and author (title). Changes for name and id will be ignored. The update happens selectively: To leave a property unchanged, set it to None. To delete existing data, use an empty string.

PARAMETERS:
info (dict) – a dictionary compatible with the info property (see below). All entries must be strings. If this argument is not a dictionary, the other arguments are used instead – else they are ignored.

content (str) – (new in v1.16.10) see description in info.

title (str) – (new in v1.16.10) see description in info.

creationDate (str) – (new in v1.16.10) date of annot creation. If given, should be in PDF datetime format.

modDate (str) – (new in v1.16.10) date of last modification. If given, should be in PDF datetime format.

subject (str) – (new in v1.16.10) see description in info.

set_line_ends(start, end)
Sets an annotation’s line ending styles. Each of these annotation types is defined by a list of points which are connected by lines. The symbol identified by start is attached to the first point, and end to the last point of this list. For unsupported annotation types, a no-operation with a warning message results.

Note

While ‘FreeText’, ‘Line’, ‘PolyLine’, and ‘Polygon’ annotations can have these properties, (Py-) MuPDF does not support line ends for ‘FreeText’, because the call-out variant of it is not supported.

(Changed in v1.16.16) Some symbols have an interior area (diamonds, circles, squares, etc.). By default, these areas are filled with the fill color of the annotation. If this is None, then white is chosen. The fill_color argument of Annot.update() can now be used to override this and give line end symbols their own fill color.

PARAMETERS:
start (int) – The symbol number for the first point.

end (int) – The symbol number for the last point.

set_oc(xref)
Set the annotation’s visibility using PDF optional content mechanisms. This visibility is controlled by the user interface of supporting PDF viewers. It is independent from other attributes like Annot.flags.

PARAMETERS:
xref (int) – the xref of an optional contents group (OCG or OCMD). Any previous xref will be overwritten. If zero, a previous entry will be removed. An exception occurs if the xref is not zero and does not point to a valid PDF object.

Note

This does not require executing Annot.update() to take effect.

get_oc()
Return the xref of an optional content object, or zero if there is none.

RETURNS:
zero or the xref of an OCG (or OCMD).

set_irt_xref(xref)
New in v1.19.3

Set annotation to be “In Response To” another one.

PARAMETERS:
xref (int) –

The xref of another annotation.

Note

Must refer to an existing annotation on this page. Setting this property requires no subsequent update().

set_open(value)
New in v1.18.4

Set the annotation’s Popup annotation to open or closed – or the annotation itself, if its type is ‘Text’ (“sticky note”).

PARAMETERS:
value (bool) – the desired open state.

set_popup(rect)
New in v1.18.4

Create a Popup annotation for the annotation and specify its rectangle. If the Popup already exists, only its rectangle is updated.

PARAMETERS:
rect (rect_like) – the desired rectangle.

set_opacity(value)
Set the annotation’s transparency. Opacity can also be set in Annot.update().

PARAMETERS:
value (float) – a float in range [0, 1]. Any value outside is assumed to be 1. E.g. a value of 0.5 sets the transparency to 50%.

Three overlapping ‘Circle’ annotations with each opacity set to 0.5:

_images/img-opacity.jpg
blendmode
New in v1.18.4

The annotation’s blend mode. See Adobe PDF References, page 324 for explanations.

RETURN TYPE:
str

RETURNS:
the blend mode or None.

set_blendmode(blendmode)
New in v1.16.14

Set the annotation’s blend mode. See Adobe PDF References, page 324 for explanations. The blend mode can also be set in Annot.update().

PARAMETERS:
blendmode (str) – set the blend mode. Use Annot.update() to reflect this in the visual appearance. For predefined values see PDF Standard Blend Modes. Use PDF_BM_Normal to remove a blend mode.

set_name(name)
New in version 1.16.0

Change the name field of any annotation type. For ‘FileAttachment’ and ‘Text’ annotations, this is the icon name, for ‘Stamp’ annotations the text in the stamp. The visual result (if any) depends on your PDF viewer. See also Annotation Icons in MuPDF.

PARAMETERS:
name (str) – the new name.

Caution

If you set the name of a ‘Stamp’ annotation, then this will not change the rectangle, nor will the text be layouted in any way. If you choose a standard text from Stamp Annotation Icons (the exact name piece after "STAMP_"), you should receive the original layout. An arbitrary text will not be changed to upper case, but be written in font “Times-Bold” as is, horizontally centered in one line and be shortened to fit. To get your text fully displayed, its length using fontsize 20 must not exceed 190 points. So please make sure that the following inequality is true: pymupdf.get_text_length(text, fontname="tibo", fontsize=20) <= 190.

set_rect(rect)
Change the rectangle of an annotation. The annotation can be moved around and both sides of the rectangle can be independently scaled. However, the annotation appearance will never get rotated, flipped or sheared. This method only affects certain annotation types [2] and will lead to a message on Python’s sys.stderr in other cases. No exception will be raised, but False will be returned.

PARAMETERS:
rect (rect_like) – the new rectangle of the annotation (finite and not empty). E.g. using a value of annot.rect + (5, 5, 5, 5) will shift the annot position 5 pixels to the right and downwards.

Note

You need not invoke Annot.update() for activation of the effect.

set_rotation(angle)
Set the rotation of an annotation. This rotates the annotation rectangle around its center point. Then a new annotation rectangle is calculated from the resulting quad.

PARAMETERS:
angle (int) – rotation angle in degrees. Arbitrary values are possible, but will be clamped to the interval [0, 360).

Note

You must invoke Annot.update() to activate the effect.

For PDF_ANNOT_FREE_TEXT, only one of the values 0, 90, 180 and 270 is possible and will rotate the text inside the current rectangle (which remains unchanged). Other values are silently ignored and replaced by 0.

Otherwise, only the following Annotation Types can be rotated: ‘Square’, ‘Circle’, ‘Caret’, ‘Text’, ‘FileAttachment’, ‘Ink’, ‘Line’, ‘Polyline’, ‘Polygon’, and ‘Stamp’. For all others the method is a no-op.

set_border(border=None, width=None, style=None, dashes=None, clouds=None)
Changed in version 1.16.9: Allow specification without using a dictionary. The direct parameters are used if border is not a dictionary.

Changed in version 1.22.5: Support of the “cloudy” border effect.

PDF only: Change border width, dashing, style and cloud effect. See the Annot.border attribute for more details.

PARAMETERS:
border (dict) – a dictionary as returned by the border property, with keys “width” (float), “style” (str), “dashes” (sequence) and clouds (int). Omitted keys will leave the resp. property unchanged. Set the border argument to None (the default) to use the other arguments.

width (float) – A non-negative value will change the border line width.

style (str) – A value other than None will change this border property.

dashes (sequence) – All items of the sequence must be integers, otherwise the parameter is ignored. To remove dashing use: dashes=[]. If dashes is a non-empty sequence, “style” will automatically be set to “D” (dashed).

clouds (int) – A value >= 0 will change this property. Use clouds=0 to remove the cloudy appearance completely. Only annotation types ‘Square’, ‘Circle’, and ‘Polygon’ are supported with this property.

set_flags(flags)
Changes the annotation flags. Use the | operator to combine several.

PARAMETERS:
flags (int) – an integer specifying the required flags.

set_colors(colors=None, stroke=None, fill=None)
Changed in version 1.16.9: Allow colors to be directly set. These parameters are used if colors is not a dictionary.

Changes the “stroke” and “fill” colors for supported annotation types – not all annotations accept both.

PARAMETERS:
colors (dict) – a dictionary containing color specifications. For accepted dictionary keys and values see below. The most practical way should be to first make a copy of the colors property and then modify this dictionary as required.

stroke (sequence) – see above.

fill (sequence) – see above.

Changed in v1.18.5: To completely remove a color specification, use an empty sequence like []. If you specify None, an existing specification will not be changed.

delete_responses()
New in version 1.16.12

Delete annotations referring to this one. This includes any ‘Popup’ annotations and all annotations responding to it.

update(opacity=None, blend_mode=None, fontsize=0, text_color=None, border_color=None, fill_color=None, cross_out=True, rotate=-1)
Synchronize the appearance of an annotation with its properties after relevant changes.

You can safely omit this method only for the following changes:

Annot.set_rect()

Annot.set_flags()

Annot.set_oc()

Annot.update_file()

Annot.set_info() (except any changes to “content”)

All arguments are optional. (Changed in v1.16.14) Blend mode and opacity are applicable to all annotation types. The other arguments are mostly special use, as described below.

Color specifications may be made in the usual format used in PuMuPDF as sequences of floats ranging from 0.0 to 1.0 (including both). The sequence length must be 1, 3 or 4 (supporting GRAY, RGB and CMYK colorspaces respectively). For GRAY, just a float is also acceptable.

PARAMETERS:
opacity (float) – (new in v1.16.14) valid for all annotation types: change or set the annotation’s transparency. Valid values are 0 <= opacity < 1.

blend_mode (str) – (new in v1.16.14) valid for all annotation types: change or set the annotation’s blend mode. For valid values see PDF Standard Blend Modes.

fontsize (float) – change fontsize of the text. ‘FreeText’ annotations only.

text_color (sequence,float) – change the text color. ‘FreeText’ annotations only.

border_color (sequence,float) – change the border color. ‘FreeText’ annotations only.

fill_color (sequence,float) –

the fill color.

’Line’, ‘Polyline’, ‘Polygon’ annotations: use it to give applicable line end symbols a fill color other than that of the annotation (changed in v1.16.16).

cross_out (bool) – (new in v1.17.2) add two diagonal lines to the annotation rectangle. ‘Redact’ annotations only. If not desired, False must be specified even if the annotation was created with False.

rotate (int) – new rotation value. Default (-1) means no change. Supports ‘FreeText’ and several other annotation types (see Annot.set_rotation()), [1]. Only choose 0, 90, 180, or 270 degrees for ‘FreeText’. Otherwise any integer is acceptable.

RETURN TYPE:
bool

Note

Using this method inside a Page.annots() loop is not recommended! This is because most annotation updates require the owning page to be reloaded – which cannot be done inside this loop. Please use the example coding pattern given in the documentation of this generator.

file_info
Basic information of the annot’s attached file.

RETURN TYPE:
dict

RETURNS:
a dictionary with keys filename, ufilename, desc (description), size (uncompressed file size), length (compressed length) for FileAttachment annot types, else None.

get_file()
Returns attached file content.

RETURN TYPE:
bytes

RETURNS:
the content of the attached file.

update_file(buffer=None, filename=None, ufilename=None, desc=None)
Updates the content of an attached file. All arguments are optional. No arguments lead to a no-op.

PARAMETERS:
buffer (bytes|bytearray|BytesIO) –

the new file content. Omit to only change meta-information.

(Changed in version 1.14.13) io.BytesIO is now also supported.

filename (str) – new filename to associate with the file.

ufilename (str) – new unicode filename to associate with the file.

desc (str) – new description of the file content.

get_sound()
Return the embedded sound of an audio annotation.

RETURN TYPE:
dict

RETURNS:
the sound audio file and accompanying properties. These are the possible dictionary keys, of which only “rate” and “stream” are always present.

Key

Description

rate

(float, requ.) samples per second

channels

(int, opt.) number of sound channels

bps

(int, opt.) bits per sample value per channel

encoding

(str, opt.) encoding format: Raw, Signed, muLaw, ALaw

compression

(str, opt.) name of compression filter

stream

(bytes, requ.) the sound file content

opacity
The annotation’s transparency. If set, it is a value in range [0, 1]. The PDF default is 1. However, in an effort to tell the difference, we return -1.0 if not set.

RETURN TYPE:
float

parent
The owning page object of the annotation.

RETURN TYPE:
Page

rotation
The annot rotation.

RETURN TYPE:
int

RETURNS:
a value [-1, 359]. If rotation is not at all, -1 is returned (and implies a rotation angle of 0). Other possible values are normalized to some value value 0 <= angle < 360.

rect
The rectangle containing the annotation.

RETURN TYPE:
Rect

next
The next annotation on this page or None.

RETURN TYPE:
Annot

type
A number and one or two strings describing the annotation type, like [2, ‘FreeText’, ‘FreeTextCallout’]. The second string entry is optional and may be empty. See the appendix Annotation Types for a list of possible values and their meanings.

RETURN TYPE:
list

info
A dictionary containing various information. All fields are optional strings. For information items not provided, an empty string is returned.

name – e.g. for ‘Stamp’ annotations it will contain the stamp text like “Sold” or “Experimental”, for other annot types you will see the name of the annot’s icon here (“PushPin” for FileAttachment).

content – a string containing the text for type Text and FreeText annotations. Commonly used for filling the text field of annotation pop-up windows.

title – a string containing the title of the annotation pop-up window. By convention, this is used for the annotation author.

creationDate – creation timestamp.

modDate – last modified timestamp.

subject – subject.

id – (new in version 1.16.10) a unique identification of the annotation. This is taken from PDF key /NM. Annotations added by PyMuPDF will have a unique name, which appears here.

RETURN TYPE:
dict

flags
An integer whose low order bits contain flags for how the annotation should be presented.

RETURN TYPE:
int

line_ends
A pair of integers specifying start and end symbol of annotations types ‘FreeText’, ‘Line’, ‘PolyLine’, and ‘Polygon’. None if not applicable. For possible values and descriptions in this list, see the Adobe PDF References, table 1.76 on page 400.

RETURN TYPE:
tuple

vertices
A list containing a variable number of point (“vertices”) coordinates (each given by a pair of floats) for various types of annotations:

‘Line’ – the starting and ending coordinates (2 float pairs).

‘FreeText’ – 2 or 3 float pairs designating the starting, the (optional) knee point, and the ending coordinates.

‘PolyLine’ / ‘Polygon’ – the coordinates of the edges connected by line pieces (n float pairs for n points).

text markup annotations – 4 float pairs specifying the QuadPoints of the marked text span (see Adobe PDF References, page 403).

‘Ink’ – list of one to many sublists of vertex coordinates. Each such sublist represents a separate line in the drawing.

RETURN TYPE:
list

colors
dictionary of two lists of floats in range 0 <= float <= 1 specifying the “stroke” and the interior (“fill”) colors. The stroke color is used for borders and everything that is actively painted or written (“stroked”). The fill color is used for the interior of objects like line ends, circles and squares. The lengths of these lists implicitly determine the colorspaces used: 1 = GRAY, 3 = RGB, 4 = CMYK. So “[1.0, 0.0, 0.0]” stands for RGB color red. Both lists can be empty if no color is specified.

RETURN TYPE:
dict

xref
The PDF xref.

RETURN TYPE:
int

irt_xref
The PDF xref of an annotation to which this one responds. Return zero if this is no response annotation.

RETURN TYPE:
int

popup_xref
The PDF xref of the associated Popup annotation. Zero if non-existent.

RETURN TYPE:
int

has_popup
Whether the annotation has a Popup annotation.

RETURN TYPE:
bool

is_open
Whether the annotation’s Popup is open – or the annotation itself (‘Text’ annotations only).

RETURN TYPE:
bool

popup_rect
The rectangle of the associated Popup annotation. Infinite rectangle if non-existent.

RETURN TYPE:
Rect

rect_delta
A tuple of four floats representing the /RD entry of the annotation. The four numbers describe the numerical differences (left, top, -right, -bottom) between two rectangles: the rect of the annotation and a rectangle contained within that rectangle. If the entry is missing, this property is (0, 0, 0, 0). If the annotation border is a normal, straight line, these numbers are typically border width divided by 2. If the annotation has a “cloudy” border, you will see the breadth of the cloud semi-circles here. In general, the numbers need not be identical. To compute the inner rectangle do a.rect + a.rect_delta.

border
A dictionary containing border characteristics. Empty if no border information exists. The following keys may be present:

width – a float indicating the border thickness in points. The value is -1.0 if no width is specified.

dashes – a sequence of integers specifying a line dashing pattern. [] means no dashes, [n] means equal on-off lengths of n points, longer lists will be interpreted as specifying alternating on-off length values. See the Adobe PDF References page 126 for more details.

style – 1-byte border style: “S” (Solid) = solid line surrounding the annotation, “D” (Dashed) = dashed line surrounding the annotation, the dash pattern is specified by the dashes entry, “B” (Beveled) = a simulated embossed rectangle that appears to be raised above the surface of the page, “I” (Inset) = a simulated engraved rectangle that appears to be recessed below the surface of the page, “U” (Underline) = a single line along the bottom of the annotation rectangle.

clouds – an integer indicating a “cloudy” border, where n is an integer -1 <= n <= 2. A value n = 0 indicates a straight line (no clouds), 1 means small and 2 means large semi-circles, mimicking the cloudy appearance. If -1, then no specification is present.

RETURN TYPE:
dict

# How to extract Images : No-PDF Documents

In contrast to the previous sections, this section deals with extracting images contained in documents, so they can be displayed as part of one or more pages.

If you want to recreate the original image in file form or as a memory area, you have basically two options:

1. Convert your document to a PDF, and then use one of the PDF-only extraction methods. This snippet will convert a document to PDF:

        pdfbytes = doc.convert_to_pdf()  # this a bytes object
        pdf = pymupdf.open("pdf", pdfbytes)  # open it as a PDF document
        # now use 'pdf' like any PDF document


2. Use Page.get_text() with the “dict” parameter. This works for all document types. It will extract all text and images shown on the page, formatted as a Python dictionary. Every image will occur in an image block, containing meta information and the binary image data. For details of the dictionary’s structure, see TextPage. The method works equally well for PDF files. This creates a list of all images shown on a page:

        d = page.get_text("dict")
        blocks = d["blocks"]  # the list of block dictionaries
        imgblocks = [b for b in blocks if b["type"] == 1]
        pprint(imgblocks[0])
        {'bbox': (100.0, 135.8769989013672, 300.0, 364.1230163574219),
        'bpc': 8,
        'colorspace': 3,
        'ext': 'jpeg',
        'height': 501,
        'image': b'\xff\xd8\xff\xe0\x00\x10JFIF\...',  # CAUTION: LARGE!
        'size': 80518,
        'transform': (200.0, 0.0, -0.0, 228.2460174560547, 100.0, 135.8769989013672),
        'type': 1,
        'width': 439,
        'xres': 96,
        'yres': 96}


# How to Extract Images: PDF Documents

Like any other “object” in a PDF, images are identified by a cross reference number (xref, an integer). If you know this number, you have two ways to access the image’s data:

1. Create a Pixmap of the image with instruction pix = pymupdf.Pixmap(doc, xref). This method is very fast (single digit micro-seconds). The pixmap’s properties (width, height, …) will reflect the ones of the image. In this case there is no way to tell which image format the embedded original has.

2. Extract the image with img = doc.extract_image(xref). This is a dictionary containing the binary image data as img[“image”]. A number of meta data are also provided – mostly the same as you would find in the pixmap of the image. The major difference is string img[“ext”], which specifies the image format: apart from “png”, strings like “jpeg”, “bmp”, “tiff”, etc. can also occur. Use this string as the file extension if you want to store to disk. The execution speed of this method should be compared to the combined speed of the statements pix = pymupdf.Pixmap(doc, xref);pix.tobytes(). If the embedded image is in PNG format, the speed of Document.extract_image() is about the same (and the binary image data are identical). Otherwise, this method is thousands of times faster, and the image data is much smaller.

The question remains: “How do I know those ‘xref’ numbers of images?”. There are two answers to this:

a. “Inspect the page objects:” Loop through the items of Page.get_images(). It is a list of list, and its items look like [xref, smask, …], containing the xref of an image. This xref can then be used with one of the above methods. Use this method for valid (undamaged) documents. Be wary however, that the same image may be referenced multiple times (by different pages), so you might want to provide a mechanism avoiding multiple extracts.

b. “No need to know:” Loop through the list of all xrefs of the document and perform a Document.extract_image() for each one. If the returned dictionary is empty, then continue – this xref is no image. Use this method if the PDF is damaged (unusable pages). Note that a PDF often contains “pseudo-images” (“stencil masks”) with the special purpose of defining the transparency of some other image. You may want to provide logic to exclude those from extraction. Also have a look at the next section.

# How to Handle Image Masks
Some images in PDFs are accompanied by image masks. In their simplest form, masks represent alpha (transparency) bytes stored as separate images. In order to reconstruct the original of an image, which has a mask, it must be “enriched” with transparency bytes taken from its mask.

Whether an image does have such a mask can be recognized in one of two ways in PyMuPDF:

An item of Document.get_page_images() has the general format (xref, smask, ...), where xref is the image’s xref and smask, if positive, then it is the xref of a mask.

The (dictionary) results of Document.extract_image() have a key “smask”, which also contains any mask’s xref if positive.

If smask == 0 then the image encountered via xref can be processed as it is.

To recover the original image using PyMuPDF, the procedure depicted as follows must be executed:

_images/img-stencil.jpg


        pix1 = pymupdf.Pixmap(doc.extract_image(xref)["image"])    # (1) pixmap of image w/o alpha
        mask = pymupdf.Pixmap(doc.extract_image(smask)["image"])   # (2) mask pixmap
        pix = pymupdf.Pixmap(pix1, mask)                           # (3) copy of pix1, image mask added

        
Step (1) creates a pixmap of the basic image. Step (2) does the same with the image mask. Step (3) adds an alpha channel and fills it with transparency information.

The scripts extract-from-pages.py, and extract-from-xref.py above also contain this logic.



# How to Create Vector Images
The usual way to create an image from a document page is Page.get_pixmap(). A pixmap represents a raster image, so you must decide on its quality (i.e. resolution) at creation time. It cannot be changed later.

PyMuPDF also offers a way to create a vector image of a page in SVG format (scalable vector graphics, defined in XML syntax). SVG images remain precise across zooming levels (of course with the exception of any raster graphic elements embedded therein).

Instruction svg = page.get_svg_image(matrix=pymupdf.Identity) delivers a UTF-8 string svg which can be stored with extension “.svg”.



# How to Convert Images
Just as a feature among others, PyMuPDF’s image conversion is easy. It may avoid using other graphics packages like PIL/Pillow in many cases.

Notwithstanding that interfacing with Pillow is almost trivial.

### Input Formats           Output Formats          Description

        BMP                 .                   Windows Bitmap

        JPEG                JPEG                Joint Photographic Experts Group

        JXR                 .                   JPEG Extended Range

        JPX/JP2             .                   JPEG 2000

        GIF                 .                   Graphics Interchange Format

        TIFF                .                   Tagged Image File Format

        PNG                 PNG                 Portable Network Graphics

        PNM                 PNM                 Portable Anymap

        PGM                 PGM                 Portable Graymap

        PBM                 PBM                 Portable Bitmap

        PPM                 PPM                 Portable Pixmap

        PAM                 PAM                 Portable Arbitrary Map

        .                   PSD                 Adobe Photoshop Document

        .                   PS                  Adobe Postscript


The general scheme is just the following two lines:

pix = pymupdf.Pixmap("input.xxx")  # any supported input format
pix.save("output.yyy")  # any supported output format
Remarks

The input argument of pymupdf.Pixmap(arg) can be a file or a bytes / io.BytesIO object containing an image.

Instead of an output file, you can also create a bytes object via pix.tobytes(“yyy”) and pass this around.

As a matter of course, input and output formats must be compatible in terms of colorspace and transparency. The Pixmap class has batteries included if adjustments are needed.

Note

Convert JPEG to Photoshop:

pix = pymupdf.Pixmap("myfamily.jpg")
pix.save("myfamily.psd")
Note

Convert JPEG to Tkinter PhotoImage. Any RGB / no-alpha image works exactly the same. Conversion to one of the Portable Anymap formats (PPM, PGM, etc.) does the trick, because they are supported by all Tkinter versions:

import tkinter as tk
pix = pymupdf.Pixmap("input.jpg")  # or any RGB / no-alpha image
tkimg = tk.PhotoImage(data=pix.tobytes("ppm"))
Note

Convert PNG with alpha to Tkinter PhotoImage. This requires removing the alpha bytes, before we can do the PPM conversion:

import tkinter as tk
pix = pymupdf.Pixmap("input.png")  # may have an alpha channel
if pix.alpha:  # we have an alpha channel!
    pix = pymupdf.Pixmap(pix, 0)  # remove it
tkimg = tk.PhotoImage(data=pix.tobytes("ppm"))